In [1]:
import pandas as pd
import numpy as np
import pypsa


In [2]:
n = pypsa.Network()

# Define hourly snapshots for full 2027
n.set_snapshots(pd.date_range("2027-01-01 00:00", "2027-12-31 23:00", freq="h"))

print("✅ Network initialized with snapshots:", len(n.snapshots))

✅ Network initialized with snapshots: 8760


In [3]:
# --------------------------------------------------
# 1️⃣ Load heat demand data (Esbjerg only)
# --------------------------------------------------
file_path = "Master_data.xlsx"
sheet_name = "Heat demand"

df_heat = pd.read_excel(file_path, sheet_name=sheet_name)
df_heat["Time"] = pd.to_datetime(df_heat["Time"])
df_heat.set_index("Time", inplace=True)

# ✅ Use "Esbjerg" column instead of "Total"
heat_demand = df_heat["Total"].rename("Heat_Demand_MW")

print("✅ Loaded heat demand data (Esbjerg only)")
print("Time range:", heat_demand.index.min(), "→", heat_demand.index.max())
print("Entries:", len(heat_demand))
print(f"Peak demand: {heat_demand.max():.2f} MW")
print(f"Annual total: {heat_demand.sum():,.0f} MWh")
print("\nFirst few values:")
print(heat_demand.head())

✅ Loaded heat demand data (Esbjerg only)
Time range: 2027-01-01 00:00:00 → 2027-12-31 23:00:00
Entries: 8760
Peak demand: 330.79 MW
Annual total: 1,236,001 MWh

First few values:
Time
2027-01-01 00:00:00    165.60
2027-01-01 01:00:00    171.68
2027-01-01 02:00:00    166.27
2027-01-01 03:00:00    166.51
2027-01-01 04:00:00    169.63
Name: Heat_Demand_MW, dtype: float64


In [4]:
# Align to 2027 hourly index (if not already)
heat_demand = heat_demand.reindex(n.snapshots, method="nearest").fillna(method="ffill")

print("✅ Heat demand aligned to network snapshots")
print("Range:", heat_demand.index.min(), "→", heat_demand.index.max())
print("Sample:\n", heat_demand.head())


✅ Heat demand aligned to network snapshots
Range: 2027-01-01 00:00:00 → 2027-12-31 23:00:00
Sample:
 snapshot
2027-01-01 00:00:00    165.60
2027-01-01 01:00:00    171.68
2027-01-01 02:00:00    166.27
2027-01-01 03:00:00    166.51
2027-01-01 04:00:00    169.63
Freq: h, Name: Heat_Demand_MW, dtype: float64


C:\Users\elshq\AppData\Local\Temp\ipykernel_29636\3910527636.py:2: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [5]:
# --------------------------------------------------
# 2️⃣ Load electricity price data (2024 sheet → map sequentially to 2027)
#     Your sheet starts on Jan 2, 2024 and has exactly 8760 rows.
#     We ignore the 'date' column and map row order to 2027 snapshots.
# --------------------------------------------------
sheet_price = "Electricity price 2024"
df_price = pd.read_excel(file_path, sheet_name=sheet_price)

# Rename defensively (adjust if your column names differ)
df_price.columns = [str(c).strip() for c in df_price.columns]
price_col = [c for c in df_price.columns if "DK1" in c or "Price" in c or "DKK" in c][0]

# Extract values as float array
price_vals = pd.to_numeric(df_price[price_col], errors="coerce").to_numpy()

# Sanity check length
if len(price_vals) != len(n.snapshots):
    raise ValueError(
        f"Price rows ({len(price_vals)}) ≠ snapshots ({len(n.snapshots)}). "
        "Expected 8760 rows (since 2027 is non-leap)."
    )

# ✅ Sequential mapping: first row → 2027-01-01 00:00, etc.
electricity_price = pd.Series(price_vals, index=n.snapshots, name="Electricity_Price")

print("✅ Electricity price mapped sequentially to 2027 snapshots")
print("Range:", electricity_price.index.min(), "→", electricity_price.index.max())
print(electricity_price.head(12))


✅ Electricity price mapped sequentially to 2027 snapshots
Range: 2027-01-01 00:00:00 → 2027-12-31 23:00:00
snapshot
2027-01-01 00:00:00    228.039993
2027-01-01 01:00:00    149.619995
2027-01-01 02:00:00    231.320007
2027-01-01 03:00:00    137.089996
2027-01-01 04:00:00     82.599998
2027-01-01 05:00:00    102.730003
2027-01-01 06:00:00    355.070007
2027-01-01 07:00:00    433.119995
2027-01-01 08:00:00    489.549988
2027-01-01 09:00:00    482.549988
2027-01-01 10:00:00    514.229980
2027-01-01 11:00:00    548.219971
Freq: h, Name: Electricity_Price, dtype: float64


In [6]:
# --- Carriers ---
for carrier in ["electricity", "heat", "waste", "gas", "biomass", "market", "electric_boiler", "heat_pump", "wte_chp"]:
    if carrier not in n.carriers.index:
        n.add("Carrier", carrier)

# --- Buses ---
buses = [
    ("el_bus", "electricity"),
    ("heat_bus", "heat"),
    ("waste_bus", "waste"),
    ("gas_bus", "gas"),
    ("biomass_bus", "biomass"),
    ("market_bus", "market"),
]
for name, carrier in buses:
    if name not in n.buses.index:
        n.add("Bus", name, carrier=carrier)


In [7]:
# --- Add district heating demand load ---
if "heat_demand" not in n.loads.index:
    n.add(
        "Load",
        "heat_demand",
        bus="heat_bus",
        carrier="heat",
        p_set=heat_demand.reindex(n.snapshots).fillna(method="ffill").values
    )

print("✅ Added district heating demand load")


✅ Added district heating demand load


C:\Users\elshq\AppData\Local\Temp\ipykernel_29636\64715060.py:8: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [ ]:
# --- Electricity import from market (buy) ---
if "el_import" not in n.generators.index:
    n.add(
        "Generator",
        "el_import",
        bus="el_bus",
        carrier="electricity",
        p_nom=1e6,                     # practically unlimited capacity
        marginal_cost=electricity_price,  # day-ahead price in DKK/MWh
    )

# --- Electricity export to market (sell) ---
if "sell_to_market" not in n.links.index:
    n.add(
        "Link",
        "sell_to_market",
        bus0="el_bus",      # from local grid
        bus1="market_bus",  # to external market
        efficiency=1.0,
        p_nom=1e6,
        marginal_cost=-electricity_price,  # negative => revenue from selling
        carrier="electricity",
    )

print("✅ Added electricity market import/export links")

# Fix: Apply hourly electricity prices to import generator
n.generators_t.marginal_cost["el_import"] = electricity_price.values

print("✅ Applied hourly electricity prices to el_import")
print(f"Price range: {electricity_price.min():.1f} - {electricity_price.max():.1f} DKK/MWh")


✅ Added electricity market import/export links
✅ Applied hourly electricity prices to el_import
Price range: -448.0 - 6982.4 DKK/MWh


In [9]:
print(n.generators_t.marginal_cost.head())


name                  el_import
snapshot                       
2027-01-01 00:00:00  228.039993
2027-01-01 01:00:00  149.619995
2027-01-01 02:00:00  231.320007
2027-01-01 03:00:00  137.089996
2027-01-01 04:00:00   82.599998


In [10]:
# Ensure carriers exist (silence warnings)
for c in ["wte_chp", "gas", "biomass", "waste"]:
    if c not in n.carriers.index:
        n.add("Carrier", c)

# 1) Waste supply (CHP links already carry marginal fuel cost → supply at 0)
if "waste_supply" not in n.generators.index:
    n.add("Generator", "waste_supply",
          bus="waste_bus", carrier="waste",
          p_nom=1e6, marginal_cost=0.0)

# 2) Gas supply (put gas fuel price here if your gas boiler link has only efficiency)
#    If you already put *full* fuel+OPEX on the gas_boiler link, keep this at 0.0
if "gas_supply" not in n.generators.index:
    n.add("Generator", "gas_supply",
          bus="gas_bus", carrier="gas",
          p_nom=1e6, marginal_cost=0.0)   # or your gas fuel DKK/MWh_fuel

# 3) Biomass (woodchip) supply
#    Same rule: if biomass_boiler link holds full marginal cost, keep this at 0.0
if "biomass_supply" not in n.generators.index:
    n.add("Generator", "biomass_supply",
          bus="biomass_bus", carrier="biomass",
          p_nom=1e6, marginal_cost=0.0)   # or biomass fuel DKK/MWh_fuel


In [11]:
# --- Electric Boiler (with startup cost via unit commitment) ---
EB_EFF = 0.99
EB_P_NOM = 40.4  # MW
EB_BASE_LOAD = 0.8           # MW_el (minimum when operating)
EB_OPEX_VAR = 4.2  # DKK/MWh
EB_STARTUP_COST = 150.0      # DKK/start

# Calculate minimum load fraction
EB_P_MIN_PU = EB_BASE_LOAD / EB_P_NOM  # 0.8 / 40.0 = 0.02 (2%)

eb_cost = EB_OPEX_VAR

if "electric_boiler" not in n.links.index:
    n.add(
        "Link",
        "electric_boiler",
        bus0="el_bus",
        bus1="heat_bus",
        efficiency=EB_EFF,
        p_nom=EB_P_NOM,
        p_min_pu=EB_P_MIN_PU,        # Add minimum 2% load
        marginal_cost=eb_cost,       # hourly series
        start_up_cost=EB_STARTUP_COST,         # DKK per start
        committable=True,            # enable UC for this link
        carrier="electric_boiler",
    )

print(f"✅ Electric boiler added:")
print(f"   Electrical: {EB_BASE_LOAD:.1f} - {EB_P_NOM:.1f} MW (min {EB_P_MIN_PU:.1%})")
print(f"   Heat output: {EB_BASE_LOAD * EB_EFF:.1f} - {EB_P_NOM * EB_EFF:.1f} MW")
print(f"   Startup cost: {EB_STARTUP_COST:,.0f} DKK")

✅ Electric boiler added:
   Electrical: 0.8 - 40.4 MW (min 2.0%)
   Heat output: 0.8 - 40.0 MW
   Startup cost: 150 DKK


In [12]:
# --- Heat Pump parameters ---
HP_COP = 3.00                 # coefficient of performance
HP_P_NOM = 23.78              # MW_el (maximum electrical consumption)
HP_BASE_LOAD = 6.56           # MW_el (minimum electrical consumption when ON)
HP_OPEX_VAR = 8.0             # DKK/MWh
HP_STARTUP_COST = 2500.0      # DKK/start

# Calculate minimum load as fraction of maximum
HP_P_MIN_PU = HP_BASE_LOAD / HP_P_NOM  # = 6.56 / 20.50 = 0.32 (32% minimum)

# Marginal cost = electricity price / COP + variable OPEX
hp_cost = HP_OPEX_VAR

# --- Add Heat Pump Link ---
if "heat_pump" not in n.links.index:
    n.add(
        "Link",
        "heat_pump",
        bus0="el_bus",              # consumes electricity
        bus1="heat_bus",            # produces heat
        efficiency=HP_COP,          # heat out per unit electricity in
        p_nom=HP_P_NOM,             # MW_th
        p_min_pu=HP_P_MIN_PU,       # Add minimum load constraint
        marginal_cost=hp_cost,      # hourly time-series cost
        start_up_cost=HP_STARTUP_COST,
        committable=True,           # enables start-up cost behaviour
        carrier="heat_pump"
    )

print(f"✅ Heat pump: {HP_P_NOM:.2f} MW_el input (min {HP_BASE_LOAD:.2f} MW = {HP_P_MIN_PU:.1%})")
print(f"   Heat output: {HP_P_NOM * HP_COP:.1f} MW (min {HP_BASE_LOAD * HP_COP:.1f} MW)")


✅ Heat pump: 23.78 MW_el input (min 6.56 MW = 27.6%)


   Heat output: 71.3 MW (min 19.7 MW)


In [13]:
# --- Natural Gas Boiler (fixed cost conversion) ---
GB_EFF = 0.80
GB_MAX_HEAT = 60.00  # MW_heat
GB_P_NOM = GB_MAX_HEAT / GB_EFF  # MW_fuel (75.00 MW)
GB_P_MIN = 9.7
GB_OPEX_VAR = 41.7        # DKK/MWh_heat
GB_STARTUP_COST = 2500.0  # DKK/start
GB_FUEL_COST = 238.0      # DKK/GJ
GB_RAMP_LIMIT = 0.20              # 20% per hour

# Convert DKK/GJ -> DKK/MWh_fuel (1 MWh = 3.6 GJ)
GB_FUEL_COST_DKK_per_MWh_fuel = GB_FUEL_COST * 3.6               # 856.8

# Heat-basis marginal cost
GB_MARGINAL_COST = GB_FUEL_COST_DKK_per_MWh_fuel / GB_EFF + GB_OPEX_VAR  # ~1112.5

if "gas_boiler" not in n.links.index:
    n.add(
        "Link", "gas_boiler",
        bus0="gas_bus", bus1="heat_bus",
        efficiency=GB_EFF,
        p_nom=GB_P_NOM,
        p_min_pu=GB_P_MIN / GB_P_NOM,
        marginal_cost=GB_MARGINAL_COST,
        start_up_cost=GB_STARTUP_COST,
        committable=True,
        carrier="gas"
    )

print(f"✅ Gas Boiler cost ≈ {GB_MARGINAL_COST:.1f} DKK/MWh_heat (fuel {GB_FUEL_COST*3.6:.1f} per MWh_fuel)")


✅ Gas Boiler cost ≈ 1112.7 DKK/MWh_heat (fuel 856.8 per MWh_fuel)


In [14]:
# --- Biomass (Woodchip) Boiler ---
BB_EFF = 0.80                    # efficiency
BB_MAX_HEAT = 60.8              # MW_heat (max)
BB_P_NOM = BB_MAX_HEAT / BB_EFF  # MW_fuel (max)
BB_P_MIN = 21.30                 # MW_th (min)
BB_OPEX_VAR = 19.30              # DKK/MWh_heat
BB_STARTUP_COST = 100000.0      # DKK/start
BB_FUEL_COST = 68.0              # DKK/GJ
BB_RAMP_LIMIT = 0.20             # 20% per hour

# DKK/GJ -> DKK/MWh_fuel (1 MWh = 3.6 GJ)
BB_FUEL_COST_per_MWh_fuel = BB_FUEL_COST * 3.6          # 244.8

# Heat-basis marginal cost
BB_MARGINAL_COST = BB_FUEL_COST_per_MWh_fuel / BB_EFF + BB_OPEX_VAR  # ≈ 325.3

if "biomass_boiler" not in n.links.index:
    n.add(
        "Link", "biomass_boiler",
        bus0="biomass_bus", bus1="heat_bus",
        efficiency=BB_EFF,
        p_nom=BB_P_NOM,
        p_min_pu=BB_P_MIN / BB_P_NOM,    # ≈ 0.355
        marginal_cost=BB_MARGINAL_COST,
        start_up_cost=BB_STARTUP_COST,
        committable=True,
        carrier="biomass"
    )

print(f"✅ Biomass boiler added | η={BB_EFF}, p_nom={BB_P_NOM} MW, cost≈{BB_MARGINAL_COST:.1f} DKK/MWh_heat")


✅ Biomass boiler added | η=0.8, p_nom=75.99999999999999 MW, cost≈325.3 DKK/MWh_heat


In [15]:
# --- Thermal Storage (Hot Water Tank) ---
TS_CAPACITY_MWh = 2325          # total capacity in MWh (from m.cp.del_t)
TS_E_MIN_PU = 0.10               # 10% minimum
TS_EFF = 0.95                    # round-trip efficiency (typical)
TS_STANDING_LOSS = 0.001         # 0.1%/h heat loss (reasonable default)

if "thermal_storage" not in n.stores.index:
    n.add(
        "Store", "thermal_storage",
        bus="heat_bus",
        e_cyclic=True,            # same SoC start/end (optional)
        e_nom=TS_CAPACITY_MWh,
        e_min_pu=TS_E_MIN_PU,
        standing_loss=TS_STANDING_LOSS,
        carrier="heat"
    )

print(f"✅ Thermal storage added | Capacity={TS_CAPACITY_MWh:.3f} MWh, η={TS_EFF}, min={TS_E_MIN_PU*100:.0f}%")


✅ Thermal storage added | Capacity=2325.000 MWh, η=0.95, min=10%


In [16]:
# ==============================
# WtE CHP - Parameter definition
# ==============================

# --- Efficiency splits
CHP_EFF_E = 0.30                  # electrical efficiency (el/fuel)
CHP_EFF_H = 0.60                  # thermal efficiency (heat/fuel)
CHP_TOTAL_EFF = CHP_EFF_E + CHP_EFF_H  # total useful efficiency (≈0.82)

# --- Capacities
CHP_P_HEAT_MAX = 85.43           # MW_heat (max)
CHP_P_ELEC_MAX = 65.0             # MW_el (max)
CHP_P_FUEL_MAX = CHP_P_HEAT_MAX / CHP_EFF_H   # ≈146.283 MW_fuel input cap

# --- Base-load & control values
CHP_BASE_HEAT = CHP_P_HEAT_MAX    # MW base heat production
CHP_C_LOWER = 0.14                # iso-fuel curve coefficient
CHP_RAMP_PCT = 0.30               # 30% ramp limit
CHP_FORCE_BASE = True             # whether to enforce base-load constraint

# ==============================
# 🔹 Marginal Cost Calculation
# ==============================

# Given fuel & OPEX data (from sheet)
CHP_FUEL_COST_DKK_per_ton = 365.0     # DKK / ton of waste
CHP_LHV_MJ_per_kg = 10.7              # MJ/kg
CHP_OPEX_VAR = 76.0  # DKK / MWh useful energyz

# --- Convert fuel cost to DKK/MWh_fuel
# 1 ton = 1000 kg → Energy_per_ton (MWh)
CHP_MWH_per_ton = (CHP_LHV_MJ_per_kg * 1000) / 3.6 / 1000  # MJ→MWh
CHP_FUEL_COST_DKK_per_MWh_fuel = (
    (CHP_FUEL_COST_DKK_per_ton) / CHP_MWH_per_ton
)


# --- 3️⃣ Combine into total marginal cost (fuel + variable OPEX)
CHP_MARG_COST = (
    CHP_FUEL_COST_DKK_per_MWh_fuel + CHP_OPEX_VAR
)

print(f"""
🧾 CHP marginal cost breakdown:
-----------------------------------
Fuel cost          : {CHP_FUEL_COST_DKK_per_MWh_fuel:8.1f} DKK/MWh_fuel
Variable OPEX      : {CHP_OPEX_VAR:8.1f} DKK/MWh_fuel
-----------------------------------
Total Marginal Cost: {CHP_MARG_COST:8.1f} DKK/MWh_fuel
""")

# You can now use:
# CHP_MARG_COST_FUEL = CHP_MARG_COST_DKK_per_MWh_fuel



🧾 CHP marginal cost breakdown:
-----------------------------------
Fuel cost          :    122.8 DKK/MWh_fuel
Variable OPEX      :     76.0 DKK/MWh_fuel
-----------------------------------
Total Marginal Cost:    198.8 DKK/MWh_fuel



In [17]:
# ==============================
# WtE CHP - Link Creation (CORRECTED)
# ==============================

# --- Ensure a fuel supply
if "waste_supply" not in n.generators.index:
    n.add("Generator", "waste_supply",
          bus="waste_bus", carrier="waste", p_nom=1e6, marginal_cost=0.0)

# --- CHP HEAT branch: fuel cost only
if "chp_heat" not in n.links.index:
    n.add("Link", "chp_heat",
          bus0="waste_bus", bus1="heat_bus",
          efficiency=CHP_EFF_H, 
          p_nom=CHP_P_FUEL_MAX,
          marginal_cost=CHP_MARG_COST,  # 198.8 DKK/MWh_fuel
          carrier="wte_chp")

# --- CHP ELECTRICITY branch: fuel cost MINUS electricity revenue
if "chp_el" not in n.links.index:
    # Net cost = fuel cost - (electricity price × efficiency)
    # This gives cost per MWh_fuel after accounting for electricity sales
    chp_el_net_cost = CHP_MARG_COST - electricity_price * CHP_EFF_E
    
    n.add("Link", "chp_el",
          bus0="waste_bus", bus1="el_bus",
          efficiency=CHP_EFF_E, 
          p_nom=CHP_P_FUEL_MAX,
          marginal_cost=chp_el_net_cost,  # Time-varying, often negative!
          carrier="wte_chp")

print(f"✅ CHP links created:")
print(f"   chp_heat: {CHP_MARG_COST:.1f} DKK/MWh_fuel")
print(f"   chp_el: {chp_el_net_cost.mean():.1f} DKK/MWh_fuel (avg, net of el. sales)")
print(f"   When el_price > {CHP_MARG_COST/CHP_EFF_E:.1f} DKK/MWh, chp_el is profitable!")

✅ CHP links created:
   chp_heat: 198.8 DKK/MWh_fuel
   chp_el: 40.5 DKK/MWh_fuel (avg, net of el. sales)
   When el_price > 662.7 DKK/MWh, chp_el is profitable!


In [18]:
# # ============================================================
# # CHP RAMPING (MANUAL - 30% of production)
# # ============================================================

# print("      Adding CHP ramp constraints (30% of production)...", flush=True)
# t1 = time.time()

# # Pre-compute CHP total production
# chp_heat_prod = p.loc[:, "chp_heat"] * CHP_EFF_H
# chp_el_prod = p.loc[:, "chp_el"] * CHP_EFF_E
# chp_total = chp_heat_prod + chp_el_prod

# chp_count = 0

# # Loop for CHP only (gas & biomass handled by PyPSA)
# for i in range(1, len(snapshots)):
#     if i % 2000 == 0:
#         print(f"         {i}/{len(snapshots)-1} hours", end="\r", flush=True)
    
#     t_curr = snapshots[i]
#     t_prev = snapshots[i-1]
    
#     # Skip maintenance
#     if t_curr in maintenance_hours or t_prev in maintenance_hours:
#         continue
    
#     # Ramp up: change ≤ 30% of previous production + 20 MW slack
#     m.add_constraints(chp_total.loc[t_curr] - chp_total.loc[t_prev] * 1.30 <= 20.0)
#     # Ramp down: change ≥ -30% of previous production - 20 MW slack
#     m.add_constraints(chp_total.loc[t_curr] - chp_total.loc[t_prev] * 0.70 >= -20.0)
#     chp_count += 2

# t2 = time.time()
# print(f"         {len(snapshots)-1}/{len(snapshots)-1} hours" + " "*20)
# print(f"      ✅ CHP: {chp_count:,} ramp constraints in {t2-t1:.1f}s")

# elapsed = time.time() - start_time
# print("="*70)
# print(f"✅ All constraints added in {elapsed:.1f}s")
# print("="*70)

In [19]:
# # ==============================
# # 🔹 Solve the optimization
# # ==============================

# print("=== Solving district heating network with CHP constraints ===")
# status, termination = n.optimize.solve_model(solver_name="gurobi", log_to_console=True)
# print(f"Status: {status} | Termination: {termination}")

# # Store and assign the results back into the network
# if status == "ok":
#     n.optimize.assign_solution()
#     print("✅ Optimization successful — results assigned to network.")
# else:
#     print("⚠️ Optimization terminated abnormally. Check solver log above.")


In [ ]:
# ================================================================
# 📋 SYSTEM ARCHITECTURE: Coupled Waste Heat Utilization Model
# ================================================================

print("""
╔══════════════════════════════════════════════════════════════════════════╗
║          COUPLED WASTE HEAT + BOOSTING SYSTEM ARCHITECTURE              ║
╚══════════════════════════════════════════════════════════════════════════╝

🔄 KEY DESIGN: HX and Boosters are COUPLED (not independent)

┌─────────────────────────────────────────────────────────────────────────┐
│ WASTE HEAT FLOW (Electrolyser → Boosters → DH Network)                 │
└─────────────────────────────────────────────────────────────────────────┘

   Electrolyser (1000 MW H2 production)
         │
         ├─→ H2 Output (p1): ~523 MW
         │
         └─→ Waste Heat (p2): ~276 MW @ 60°C
                  ↓
         waste_heat_bus (single energy balance point)
                  │
      ┌───────────┴───────────┬─────────────┬──────────────┐
      ↓                       ↓             ↓              ↓
   BOOSTERS              BYPASS         CHILLER      (NO separate HX!)
  (Priority!)           (Rare)        (Residual)
      │
      │ Each booster uses TWO energy inputs:
      │  • Waste heat (p2): 40°C→60°C base heating (34%, FREE)
      │  • Fuel (p0): 60°C→90°C top-up heating (66%, PAID)
      │
      └─→ Heat Output (p1): Full 40°C→90°C equivalent @ heat_bus

┌─────────────────────────────────────────────────────────────────────────┐
│ COMPONENT CONNECTIONS                                                    │
└─────────────────────────────────────────────────────────────────────────┘

Component              │ bus0 (input)    │ bus1 (output)   │ bus2 (waste heat)
──────────────────────┼─────────────────┼─────────────────┼──────────────────
Electrolyser          │ el_bus          │ h2_bus          │ waste_heat_bus (produces)
                      │                 │                 │
CHP Booster           │ waste_bus       │ heat_bus        │ waste_heat_bus (consumes)
Electric Booster      │ el_bus          │ heat_bus        │ waste_heat_bus (consumes)
Gas Booster           │ gas_bus         │ heat_bus        │ waste_heat_bus (consumes)
Biomass Booster       │ biomass_bus     │ heat_bus        │ waste_heat_bus (consumes)
                      │                 │                 │
Bypass                │ waste_heat_bus  │ (none)          │ (none)
Chiller               │ waste_heat_bus  │ el_bus          │ (none)
                      │                 │                 │
Direct Heating:       │                 │                 │
  CHP (Direct)        │ waste_bus       │ heat_bus        │ (none)
  Heat Pump           │ el_bus          │ heat_bus        │ (none)
  Biomass (Direct)    │ biomass_bus     │ heat_bus        │ (none)
  Gas (Direct)        │ gas_bus         │ heat_bus        │ (none)
  Electric (Direct)   │ el_bus          │ heat_bus        │ (none)

┌─────────────────────────────────────────────────────────────────────────┐
│ ENERGY BALANCE & PRIORITY                                               │
└─────────────────────────────────────────────────────────────────────────┘

waste_heat_bus energy balance (automatic via PyPSA):
    Electrolyser_waste_heat = Boosters + Bypass + Chiller

Heat supply priority (via marginal costs):
    1️⃣  BOOSTERS (110 MW total) - Best efficiency (ΔT=30°C for fuel portion)
    2️⃣  DIRECT HEATING - Used when boosting capacity exhausted (ΔT=50°C)

Each booster provides:
    • Base heating from waste heat: 34% of total heat (40°C→60°C) - FREE
    • Top-up from fuel: 66% of total heat (60°C→90°C) - PAID
    • Total output: Full 40°C→90°C equivalent delivered to heat_bus

Economic advantage:
    • Boosters use LESS fuel per MWh of heat (only need to cover 60°C→90°C)
    • Direct heating uses MORE fuel per MWh of heat (must cover 40°C→90°C)
    • Result: Optimizer prefers boosters when waste heat available

Chiller dynamic load:
    • Chiller electricity = (Waste_heat - Boosters_consumed) / COP
    • More boosting → Less chiller load → Lower electricity cost
    • Coupling enforced: waste heat ALWAYS used productively

┌─────────────────────────────────────────────────────────────────────────┐
│ KEY FEATURES                                                             │
└─────────────────────────────────────────────────────────────────────────┘

✅ Waste heat and boosting are COUPLED (not independent)
✅ Boosters have priority over direct heating (better efficiency)
✅ Single waste_heat_bus simplifies energy balance
✅ Chiller load varies dynamically with booster utilization
✅ Correct economic incentives: maximize waste heat use via boosting
✅ No separate HX link - coupling implicit in booster model

═══════════════════════════════════════════════════════════════════════════
""")


In [ ]:
# ==============================
# 🌡️ ELECTROLYSER + DYNAMIC THERMAL-HYDRAULIC SYSTEM
# ==============================
# This creates a complete thermal-hydraulic model with dynamic flow control
# ==============================

import pandas as pd
import numpy as np

print("\n" + "="*70)
print("🌡️  Building Dynamic Thermal-Hydraulic Model for Electrolyser System")
print("="*70)

# ================================================================
# PARAMETERS - Electrolyser & Thermal System
# ================================================================

print("\n📋 System Parameters:")

# --- Electrolyser ---
ELECTRO_P_NOM = 1000.0           # MW electrical capacity
ELECTRO_EFF_H2 = 0.5234          # H2 efficiency (MWh_H2 / MWh_el)
ELECTRO_EFF_HEAT = 0.276         # Waste heat efficiency (MW_heat / MW_el)
ELECTRO_P_MIN_PU = 0.05          # 5% minimum load
ELECTRO_STARTUP = 7460         # DKK per start
ELECTRO_RAMP_UP = 0.75              # 75% per hour
ELECTRO_RAMP_DOWN = 0.75            # 75% per hour

# --- Chiller ---
CHILLER_COP = 3.0                # Coefficient of performance
CHILLER_P_MIN_PU = 0.10          # 10% minimum when ON
CHILLER_STARTUP = 0              # DKK per start

# ⚠️ CRITICAL: Since bus0 is NOT el_bus, must include electricity cost in marginal_cost!
# Chiller consumes electricity from el_bus as bus1 (output with negative efficiency)
# PyPSA does NOT automatically include electricity price in this configuration
# We'll use TIME-VARYING electricity prices applied after chiller creation
CHILLER_ELEC_PER_THERMAL = 1.0 / CHILLER_COP  # 0.333 MWh_elec per MWh_thermal
CHILLER_OPEX_BASE = 60.0         # DKK/MWh_thermal (maintenance, fixed O&M)
# Note: Time-varying electricity cost will be added after chiller is created

# --- H2 Economics (ALL costs in electrolyser marginal_cost) ---
H2_KG_PER_MWH_ELEC = 19.11       # kg H2 per MWh electricity
H2_REVENUE = 37.30               # DKK/kg H2
DSO_TARIFF = 72.99               # DKK/MWh electricity
ELECTRO_OPEX_VAR_PER_KG = 0.67          # DKK/kg H2

# --- Thermal Circuit ---
TOTAL_FLOW = 2200.0              # kg/s total circulation
CP_WATER = 4.186                 # kJ/(kg·K)
T_ELECTRO_OUT = 60.0             # °C electrolyser outlet
T_ELECTRO_IN = 30.0              # °C electrolyser inlet (after chiller)
T_DH_RETURN = 40.0               # °C DH return temperature
T_DH_SUPPLY = 90.0               # °C DH supply temperature
HX_EFFECTIVENESS = 1          # Heat exchanger effectiveness

# --- Temperature Rise Calculations ---
# After waste heat HX: 40 + 1 × (60 - 40) = 60°C
T_DH_AFTER_HX = T_DH_RETURN + HX_EFFECTIVENESS * (T_ELECTRO_OUT - T_DH_RETURN)

# Energy fractions (proportional to temperature rise)
TOTAL_TEMP_RISE = T_DH_SUPPLY - T_DH_RETURN      # 50°C total
TEMP_RISE_FROM_WASTE = T_DH_AFTER_HX - T_DH_RETURN  # 17°C from waste heat
TEMP_RISE_FROM_FUEL = T_DH_SUPPLY - T_DH_AFTER_HX   # 33°C from fuel

FRACTION_FROM_WASTE = TEMP_RISE_FROM_WASTE / TOTAL_TEMP_RISE  # 0.34 (34%)
FRACTION_FROM_FUEL = TEMP_RISE_FROM_FUEL / TOTAL_TEMP_RISE    # 0.66 (66%)

# --- Boosting Allocation (from your existing system) ---
BOOST_CAPACITIES = {
    'chp_heat': 20.0,            # MW_heat for boosting
    'electric_boiler': 20.0,
    'gas_boiler': 20.0,
    'biomass_boiler': 50.0,
}

print(f"   • Electrolyser: {ELECTRO_P_NOM:.0f} MW, η_H2={ELECTRO_EFF_H2:.2%}, η_heat={ELECTRO_EFF_HEAT:.3f}")
print(f"   • Max waste heat: {ELECTRO_P_NOM * ELECTRO_EFF_HEAT:.1f} MW @ full load")
print(f"   • Chiller: COP={CHILLER_COP:.1f}")
print(f"      ├─ Elec cost: {CHILLER_ELEC_PER_THERMAL:.3f} × (time-varying spot price)")
print(f"      ├─ OPEX cost: {CHILLER_OPEX_BASE:.1f} DKK/MWh_thermal")
print(f"      └─ Total cost: Time-varying (spot price dependent)")
print(f"   • Circulation: {TOTAL_FLOW:.0f} kg/s, {T_ELECTRO_OUT:.0f}°C → {T_ELECTRO_IN:.0f}°C")
print(f"   • Total boosting capacity: {sum(BOOST_CAPACITIES.values()):.0f} MW_heat")

# ================================================================
# STEP 1: CARRIERS & BUSES
# ================================================================

print("Creating infrastructure...")

# Carriers
for carrier in ["H2", "electrolyser", "cooling", "waste_heat", "heat_bypass"]:
    if carrier not in n.carriers.index:
        n.add("Carrier", carrier)

for component in BOOST_CAPACITIES.keys():
    boost_carrier = f"{component}_boost"
    if boost_carrier not in n.carriers.index:
        n.add("Carrier", boost_carrier)

# Buses
if "h2_bus" not in n.buses.index:
    n.add("Bus", "h2_bus", carrier="H2")
    print("   ✅ h2_bus")

if "waste_heat_bus" not in n.buses.index:
    n.add("Bus", "waste_heat_bus", carrier="waste_heat")
    print("   ✅ waste_heat_bus (single bus for all waste heat operations)")

if "waste_heat_bus" not in n.buses.index:
    n.add("Bus", "waste_heat_bus", carrier="waste_heat")
    print("   ✅ waste_heat_bus")

# ================================================================

# ================================================================
# STEP 2: ELECTROLYSER
# ================================================================

print("\n⚡ Creating Electrolyser...")

# Calculate net marginal cost (includes ALL economics)
H2_REVENUE_PER_MWH_ELEC = H2_KG_PER_MWH_ELEC * H2_REVENUE        # 712.8
OPEX_PER_MWH_ELEC = ELECTRO_OPEX_VAR_PER_KG * H2_KG_PER_MWH_ELEC        # 12.8
electro_marginal_cost = DSO_TARIFF + OPEX_PER_MWH_ELEC - H2_REVENUE_PER_MWH_ELEC  # ≈ -626.0

print(f"   💰 Economics (per MWh electricity):")
print(f"      • DSO tariff: +{DSO_TARIFF:.1f} DKK")
print(f"      • Variable OPEX: +{OPEX_PER_MWH_ELEC:.1f} DKK")
print(f"      • H2 revenue: -{H2_REVENUE_PER_MWH_ELEC:.1f} DKK")
print(f"      • Net marginal cost: {electro_marginal_cost:.1f} DKK/MWh")
print(f"      • Break-even price: {-electro_marginal_cost:.1f} DKK/MWh")

# Remove old if exists
if "electrolyser" in n.links.index:
    n.links.drop("electrolyser", inplace=True)

n.add(
    "Link",
    "electrolyser",
    bus0="el_bus",
    bus1="h2_bus",
    bus2="waste_heat_bus",     # Waste heat output
    efficiency=ELECTRO_EFF_H2,
    efficiency2=ELECTRO_EFF_HEAT,
    p_nom=ELECTRO_P_NOM,
    p_min_pu=ELECTRO_P_MIN_PU,
    marginal_cost=electro_marginal_cost,
    committable=True,
    start_up_cost=ELECTRO_STARTUP,
    carrier="electrolyser"
)

print(f"   ✅ Electrolyser: {ELECTRO_P_NOM:.0f} MW, startup: {ELECTRO_STARTUP:.0f} DKK")

# ================================================================
# STEP 3: H2 STORE (ZERO marginal cost - revenue in electrolyser)
# ================================================================

print("\n💎 Creating H2 Store...")

if "h2_store" in n.stores.index:
    n.stores.drop("h2_store", inplace=True)

n.add(
    "Store",
    "h2_store",
    bus="h2_bus",
    e_nom=1e9,                  # Unlimited
    e_initial=0,
    e_cyclic=False,
    marginal_cost=0.0,          # ✅ ZERO - revenue already in electrolyser
    carrier="H2"
)

print(f"   ✅ H2 store: Unlimited capacity, zero marginal cost (revenue in electrolyser)")

# ================================================================
# STEP 3b: WASTE HEAT EXCHANGER
# ================================================================
# This link models the heat exchanger between:
# - Electrolyser cooling circuit (60°C waste heat)
# - DH return water (40°C → heated to ~57°C)
#
# The heat exchanger effectiveness determines how much of the 
# temperature difference is captured:
# T_after_HX = T_return + HX_effectiveness × (T_waste - T_return)
#            = 40 + 1 × (60 - 40) = 60°C
#
# This provides "free" preheating for the DH system, reducing
# the fuel needed by the boosters from 50°C rise to 30°C rise.

print("\n🔄 Creating Waste Heat Exchanger...")

# Calculate max waste heat capacity
WASTE_HEAT_MAX_MW = ELECTRO_P_NOM * ELECTRO_EFF_HEAT  # 276 MW

# ================================================================
# 🔄 COUPLED WASTE HEAT UTILIZATION MODEL
# ================================================================
# Waste heat HX is NOT created as a separate link.
# Instead, boosters are modeled to provide FULL heat output using:
#   - Waste heat for base heating (40°C→60°C) - FREE
#   - Fuel for top-up heating (60°C→90°C) - PAID
# This enforces coupling: waste heat ALWAYS provides base heating,
# and boosters have priority over direct heating (better efficiency).
#
# The fractions below determine energy distribution:
#   - FRACTION_FROM_WASTE = 34% (waste heat provides 17°C of 50°C rise)
#   - FRACTION_FROM_FUEL = 66% (fuel provides 33°C of 50°C rise)

# Add waste_heat carrier if not exists
if "waste_heat" not in n.carriers.index:
    n.add("Carrier", "waste_heat")

print(f"   ✅ Waste Heat Exchanger:")
print(f"      • Input: waste_heat_bus (60°C waste heat)")
print(f"      • Output: heat_bus (preheated DH water)")
print(f"      • Capacity: {WASTE_HEAT_MAX_MW:.1f} MW")
print(f"      • Effectiveness: {HX_EFFECTIVENESS:.0%}")
print(f"      • Marginal cost: 0 DKK/MWh (FREE preheating!)")

# ================================================================
# 📊 TEMPERATURE-ENERGY RELATIONSHIP SUMMARY
# ================================================================
print(f"\n📊 Temperature-Energy Relationship:")
print(f"   • DH return: {T_DH_RETURN:.0f}°C")
print(f"   • After HX: {T_DH_RETURN + HX_EFFECTIVENESS * (T_ELECTRO_OUT - T_DH_RETURN):.1f}°C")
print(f"   • DH supply: {T_DH_SUPPLY:.0f}°C")
print(f"   • Mass flow: {TOTAL_FLOW:.0f} kg/s")
print(f"   • Energy from waste: {FRACTION_FROM_WASTE:.1%} (17°C rise)")
print(f"   • Energy from fuel: {FRACTION_FROM_FUEL:.1%} (33°C rise)")



🌡️  Building Dynamic Thermal-Hydraulic Model for Electrolyser System

📋 System Parameters:
   • Electrolyser: 1000 MW, η_H2=52.34%, η_heat=0.276
   • Max waste heat: 276.0 MW @ full load
   • Chiller: COP=3.0
      ├─ Elec cost: 0.333 × (time-varying spot price)
      ├─ OPEX cost: 60.0 DKK/MWh_thermal
      └─ Total cost: Time-varying (spot price dependent)
   • Circulation: 2200 kg/s, 60°C → 30°C
   • Total boosting capacity: 110 MW_heat
Creating infrastructure...
   ✅ h2_bus
   ✅ electro_60c_bus
   ✅ dh_return_40c (DH return at 40°C)
   ✅ dh_intermediate_57c (after waste heat HX)
   ✅ dh_supply_90c (DH supply at 90°C)

⚡ Creating Electrolyser...
   💰 Economics (per MWh electricity):
      • DSO tariff: +73.0 DKK
      • Variable OPEX: +12.8 DKK
      • H2 revenue: -712.8 DKK
      • Net marginal cost: -627.0 DKK/MWh
      • Break-even price: 627.0 DKK/MWh
   ✅ Electrolyser: 1000 MW, startup: 7460 DKK

💎 Creating H2 Store...
   ✅ H2 store: Unlimited capacity, zero marginal cost (rev

In [21]:
# ================================================================
# 🔥 BOOSTING SYSTEM - INDIVIDUAL LINKS WITH WASTE HEAT
# ================================================================

print("\n🔥 Creating Boosting Links (Individual Configuration)...")

# ================================================================
# Heat Transfer Parameters (from cell 19)
# ================================================================
# Using pre-calculated values:
# - T_DH_RETURN = 40°C (DH return temperature)
# - T_DH_AFTER_HX = 57°C (after waste heat recovery)
# - T_DH_SUPPLY = 90°C (DH supply temperature)
# - FRACTION_FROM_WASTE = 0.34 (34% of heat from waste heat)
# - FRACTION_FROM_FUEL = 0.66 (66% of heat from fuel)

print(f"\n📊 Energy Balance (per MW of DH output at 90°C):")
print(f"   • DH return temp: {T_DH_RETURN:.0f}°C")
print(f"   • After waste heat HX: {T_DH_AFTER_HX:.1f}°C")
print(f"   • Final DH supply: {T_DH_SUPPLY:.0f}°C")
print(f"   • Heat from 60°C waste: {FRACTION_FROM_WASTE:.1%} (FREE!)")
print(f"   • Heat from fuel: {FRACTION_FROM_FUEL:.1%} (PAID!)")

# ================================================================
# 🆕 CHILLER OPPORTUNITY COST CALCULATION
# ================================================================
avg_elec_price = electricity_price.mean()
chiller_elec_per_waste = 1.0 / CHILLER_COP  # 0.333 MWh_elec per MWh_waste_heat
opportunity_cost_per_mwh_waste = chiller_elec_per_waste * (avg_elec_price + CHILLER_OPEX_BASE)

print(f"\n💰 Chiller Opportunity Cost (per MWh of waste heat used):")
print(f"   • Avg electricity price: {avg_elec_price:.1f} DKK/MWh")
print(f"   • Chiller OPEX: {CHILLER_OPEX_BASE:.1f} DKK/MWh_thermal")
print(f"   • Electricity per waste: {chiller_elec_per_waste:.3f} MWh_elec/MWh_waste")
print(f"   • Opportunity cost: {opportunity_cost_per_mwh_waste:.1f} DKK/MWh_waste")

boost_count = 0

# ================================================================
# 1️⃣ CHP HEAT BOOSTER
# ================================================================
if 'chp_heat' in n.links.index:
    boost_heat_mw = BOOST_CAPACITIES['chp_heat']  # 20 MW
    
    # Get original component properties
    original_fuel_bus = n.links.at['chp_heat', 'bus0']  # waste_bus
    original_efficiency = n.links.at['chp_heat', 'efficiency']
    original_marginal_cost = n.links.at['chp_heat', 'marginal_cost']
    
    # Calculate booster parameters
    fuel_for_dh = boost_heat_mw * FRACTION_FROM_FUEL / original_efficiency
    efficiency_boost = boost_heat_mw / fuel_for_dh
    waste_heat_per_fuel = (boost_heat_mw * FRACTION_FROM_WASTE) / fuel_for_dh
    efficiency2_boost = -waste_heat_per_fuel
    
    # Economic calculations
    fuel_cost_component = original_marginal_cost * FRACTION_FROM_FUEL
    chiller_savings_per_fuel = waste_heat_per_fuel * opportunity_cost_per_mwh_waste
    boost_marginal_cost = fuel_cost_component - chiller_savings_per_fuel
    
    # Remove old if exists
    if 'chp_heat_boost' in n.links.index:
        n.links.drop('chp_heat_boost', inplace=True)
    
    # Create booster link
    n.add(
        "Link",
        "chp_heat_boost",
        bus0="waste_bus",           # Fuel input (waste)
        bus1="heat_bus",            # Heat output to DH
        bus2="waste_heat_bus",      # Consumes waste heat (negative efficiency)
        efficiency=efficiency_boost,
        efficiency2=efficiency2_boost,
        p_nom=fuel_for_dh,
        marginal_cost=boost_marginal_cost,
        carrier="chp_heat_boost"
    )
    
    boost_count += 1
    print(f"\n   ✅ chp_heat_boost: {boost_heat_mw:.1f} MW DH output")

# ================================================================
# 2️⃣ ELECTRIC BOILER BOOSTER
# ================================================================
if 'electric_boiler' in n.links.index:
    boost_heat_mw = BOOST_CAPACITIES['electric_boiler']  # 20 MW
    
    original_fuel_bus = n.links.at['electric_boiler', 'bus0']  # el_bus
    original_efficiency = n.links.at['electric_boiler', 'efficiency']
    original_marginal_cost = n.links.at['electric_boiler', 'marginal_cost']
    
    fuel_for_dh = boost_heat_mw * FRACTION_FROM_FUEL / original_efficiency
    efficiency_boost = boost_heat_mw / fuel_for_dh
    waste_heat_per_fuel = (boost_heat_mw * FRACTION_FROM_WASTE) / fuel_for_dh
    efficiency2_boost = -waste_heat_per_fuel
    
    fuel_cost_component = original_marginal_cost * FRACTION_FROM_FUEL
    chiller_savings_per_fuel = waste_heat_per_fuel * opportunity_cost_per_mwh_waste
    boost_marginal_cost = fuel_cost_component - chiller_savings_per_fuel
    
    if 'electric_boiler_boost' in n.links.index:
        n.links.drop('electric_boiler_boost', inplace=True)
    
    n.add(
        "Link",
        "electric_boiler_boost",
        bus0="el_bus",              # Electricity input
        bus1="heat_bus",            # Heat output to DH
        bus2="waste_heat_bus",      # Consumes waste heat (negative efficiency)
        efficiency=efficiency_boost,
        efficiency2=efficiency2_boost,
        p_nom=fuel_for_dh,
        marginal_cost=boost_marginal_cost,
        carrier="electric_boiler_boost"
    )
    
    boost_count += 1
    print(f"   ✅ electric_boiler_boost: {boost_heat_mw:.1f} MW DH output")

# ================================================================
# 3️⃣ GAS BOILER BOOSTER
# ================================================================
if 'gas_boiler' in n.links.index:
    boost_heat_mw = BOOST_CAPACITIES['gas_boiler']  # 20 MW
    
    original_fuel_bus = n.links.at['gas_boiler', 'bus0']  # gas_bus
    original_efficiency = n.links.at['gas_boiler', 'efficiency']
    original_marginal_cost = n.links.at['gas_boiler', 'marginal_cost']
    
    fuel_for_dh = boost_heat_mw * FRACTION_FROM_FUEL / original_efficiency
    efficiency_boost = boost_heat_mw / fuel_for_dh
    waste_heat_per_fuel = (boost_heat_mw * FRACTION_FROM_WASTE) / fuel_for_dh
    efficiency2_boost = -waste_heat_per_fuel
    
    fuel_cost_component = original_marginal_cost * FRACTION_FROM_FUEL
    chiller_savings_per_fuel = waste_heat_per_fuel * opportunity_cost_per_mwh_waste
    boost_marginal_cost = fuel_cost_component - chiller_savings_per_fuel
    
    if 'gas_boiler_boost' in n.links.index:
        n.links.drop('gas_boiler_boost', inplace=True)
    
    n.add(
        "Link",
        "gas_boiler_boost",
        bus0="gas_bus",             # Gas input
        bus1="heat_bus",            # Heat output to DH
        bus2="waste_heat_bus",      # Consumes waste heat (negative efficiency)
        efficiency=efficiency_boost,
        efficiency2=efficiency2_boost,
        p_nom=fuel_for_dh,
        marginal_cost=boost_marginal_cost,
        carrier="gas_boiler_boost"
    )
    
    boost_count += 1
    print(f"   ✅ gas_boiler_boost: {boost_heat_mw:.1f} MW DH output")

# ================================================================
# 4️⃣ BIOMASS BOILER BOOSTER
# ================================================================
if 'biomass_boiler' in n.links.index:
    boost_heat_mw = BOOST_CAPACITIES['biomass_boiler']  # 50 MW
    
    original_fuel_bus = n.links.at['biomass_boiler', 'bus0']  # biomass_bus
    original_efficiency = n.links.at['biomass_boiler', 'efficiency']
    original_marginal_cost = n.links.at['biomass_boiler', 'marginal_cost']
    
    fuel_for_dh = boost_heat_mw * FRACTION_FROM_FUEL / original_efficiency
    efficiency_boost = boost_heat_mw / fuel_for_dh
    waste_heat_per_fuel = (boost_heat_mw * FRACTION_FROM_WASTE) / fuel_for_dh
    efficiency2_boost = -waste_heat_per_fuel
    
    fuel_cost_component = original_marginal_cost * FRACTION_FROM_FUEL
    chiller_savings_per_fuel = waste_heat_per_fuel * opportunity_cost_per_mwh_waste
    boost_marginal_cost = fuel_cost_component - chiller_savings_per_fuel
    
    if 'biomass_boiler_boost' in n.links.index:
        n.links.drop('biomass_boiler_boost', inplace=True)
    
    n.add(
        "Link",
        "biomass_boiler_boost",
        bus0="biomass_bus",         # Biomass input
        bus1="heat_bus",            # Heat output to DH
        bus2="waste_heat_bus",      # Consumes waste heat (negative efficiency)
        efficiency=efficiency_boost,
        efficiency2=efficiency2_boost,
        p_nom=fuel_for_dh,
        marginal_cost=boost_marginal_cost,
        carrier="biomass_boiler_boost"
    )
    
    boost_count += 1
    print(f"   ✅ biomass_boiler_boost: {boost_heat_mw:.1f} MW DH output")

print(f"\n" + "="*70)
print(f"✅ {boost_count} Booster links created (individual configuration)")
print(f"   • Each booster: bus0=fuel, bus1=heat_bus, bus2=waste_heat_bus")
print(f"   • Waste heat consumed via negative efficiency2")
print(f"   • Economic optimization includes chiller opportunity cost")
print("="*70)



🔥 Creating Boosting Links (Physics-Based Economics with Chiller Opportunity Cost)...

📊 Energy Balance (per MW of DH output at 90°C):
   • DH return temp: 40°C
   • After waste heat HX: 60.0°C
   • Final DH supply: 90°C
   • Heat from 60°C waste: 40.0% (FREE!)
   • Heat from fuel: 60.0% (PAID!)

💰 Chiller Opportunity Cost (per MWh of waste heat used):
   • Avg electricity price: 527.7 DKK/MWh
   • Chiller OPEX: 60.0 DKK/MWh_thermal
   • Total chiller cost: 587.7 DKK/MWh_thermal
   • Chiller elec per waste heat: 0.333 MWh_elec/MWh_waste
   • ✅ Opportunity cost: 195.9 DKK/MWh_waste (SAVED!)

   ✅ chp_heat_boost:
      • DH output capacity: 20.0 MW
      
      💰 Cost Breakdown (per hour at full load):
         OPTION 1 - Direct Heating (no waste heat):
         • Fuel: 33.3 MW × 198.8 = 6627 DKK/h
         • Chiller: 8.0 MW_thermal × 587.7 = 1567 DKK/h
         • TOTAL: 8194 DKK/h
         
         OPTION 2 - Boosted Heating (with waste heat):
         • Fuel: 20.0 MW × 119.3 = 2386 DK

In [22]:
# ================================================================
# STEP 5: BYPASS FOR EXCESS 60°C HEAT
# ================================================================

# Option A: Use a sink bus
if "heat_sink" not in n.buses.index:
    n.add("Bus", "heat_sink", carrier="heat_sink")

n.add(
    "Link",
    "waste_heat_bypass",
    bus0="waste_heat_bus",
    bus1="heat_sink",           # ✅ Different bus (cleaner)
    efficiency= 0.0001,
    p_nom=1000.0,
    marginal_cost=0.0,
    carrier="heat_bypass"
)

print(f"   ✅ Bypass: 1000 MW capacity (free disposal)")

print("="*70)
print("✅ Electrolyser system infrastructure complete")
print("="*70)

   ✅ Bypass: 1000 MW capacity (free disposal)
✅ Electrolyser system infrastructure complete


In [23]:
# ==============================
# 🧮 DYNAMIC CHILLER SIZING CALCULATION
# ==============================
# This calculates the required chiller capacity based on
# worst-case thermal-hydraulic scenario
# ==============================

print("\n🧮 CALCULATING DYNAMIC CHILLER REQUIREMENTS")
print("="*70)

# Parameters from boosting system
Q_ELECTRO_MAX = ELECTRO_P_NOM * ELECTRO_EFF_HEAT  # Max waste heat at 60°C
TOTAL_BOOST_HEAT_CAP = sum(BOOST_CAPACITIES.values())  # Total DH heat capacity from boosters

# Calculate max 60°C consumption by boosters
# Each MW of DH output needs FRACTION_FROM_WASTE / HX_EFFECTIVENESS of 60°C heat
WASTE_HEAT_PER_MW_DH = FRACTION_FROM_WASTE / HX_EFFECTIVENESS  # 0.34 / 0.85 = 0.40
MAX_60C_TO_BOOSTERS = TOTAL_BOOST_HEAT_CAP * WASTE_HEAT_PER_MW_DH

print(f"\n📊 Thermal Analysis:")
print(f"   • Max electrolyser waste heat: {Q_ELECTRO_MAX:.1f} MW @ 60°C")
print(f"   • Total booster DH capacity: {TOTAL_BOOST_HEAT_CAP:.1f} MW_heat")
print(f"   • 60°C heat per MW DH boost: {WASTE_HEAT_PER_MW_DH:.3f} MW")
print(f"   • Max 60°C to boosters: {MAX_60C_TO_BOOSTERS:.1f} MW")

# Scenario 1: Zero boosting (worst case for chiller)
print(f"\n🔍 Scenario 1: Zero Boosting (All Waste Heat to Chiller)")
Q_to_chiller_1 = Q_ELECTRO_MAX
chiller_load_1 = Q_to_chiller_1
print(f"   • All waste heat to chiller: {Q_to_chiller_1:.1f} MW @ 60°C")
print(f"   • Chiller thermal load: {chiller_load_1:.1f} MW")

# Scenario 2: Maximum boosting (minimum to chiller)
print(f"\n🔍 Scenario 2: Maximum Boosting (High Heat Demand)")
Q_to_boosters_2 = MAX_60C_TO_BOOSTERS
Q_to_chiller_2 = Q_ELECTRO_MAX - Q_to_boosters_2
chiller_load_2 = Q_to_chiller_2
print(f"   • 60°C to boosters: {Q_to_boosters_2:.1f} MW")
print(f"   • 60°C to chiller: {Q_to_chiller_2:.1f} MW")
print(f"   • Chiller thermal load: {chiller_load_2:.1f} MW")

# Scenario 3: Partial electrolyser load (check if ever worse)
# At minimum electrolyser load with zero boosting
Q_ELECTRO_MIN = ELECTRO_P_NOM * ELECTRO_P_MIN_PU * ELECTRO_EFF_HEAT
chiller_load_3 = Q_ELECTRO_MIN
print(f"\n🔍 Scenario 3: Minimum Electrolyser Load (No Boosting)")
print(f"   • Electrolyser at {ELECTRO_P_MIN_PU*100:.0f}% load")
print(f"   • Waste heat: {Q_ELECTRO_MIN:.1f} MW @ 60°C")
print(f"   • Chiller thermal load: {chiller_load_3:.1f} MW")

# Worst case sizing
CHILLER_MW_COOLING = max(chiller_load_1, chiller_load_2, chiller_load_3)
CHILLER_P_MIN = (Q_ELECTRO_MIN / CHILLER_COP) * CHILLER_P_MIN_PU  # Min stable operation

print(f"\n🎯 CHILLER SIZING:")
print(f"   • Required cooling capacity: {CHILLER_MW_COOLING:.1f} MW_thermal")
print(f"   • COP: {CHILLER_COP:.1f}")
print(f"   • Electrical capacity: {CHILLER_MW_COOLING/CHILLER_COP:.1f} MW_elec")
print(f"   • Operating range: {CHILLER_P_MIN:.1f} - {CHILLER_MW_COOLING/CHILLER_COP:.1f} MW_elec")
print(f"   • Min load constraint: {CHILLER_P_MIN_PU*100:.0f}% of capacity")

# Validate sizing
print(f"\n✅ VALIDATION:")
if CHILLER_MW_COOLING >= Q_ELECTRO_MAX * 0.99:
    print(f"   ✅ Can handle 100% electrolyser load with zero boosting")
else:
    print(f"   ⚠️  Undersized: Only {CHILLER_MW_COOLING/Q_ELECTRO_MAX*100:.1f}% of max waste heat")

if CHILLER_MW_COOLING >= Q_to_chiller_2:
    print(f"   ✅ Can handle minimum load ({Q_to_chiller_2:.1f} MW) with max boosting")
else:
    print(f"   ⚠️  Cannot handle scenario with max boosting")

print(f"\n💡 Dynamic Range:")
print(f"   • Max thermal load: {CHILLER_MW_COOLING:.1f} MW (0% boosting)")
print(f"   • Min thermal load: {Q_to_chiller_2:.1f} MW ({MAX_60C_TO_BOOSTERS/Q_ELECTRO_MAX*100:.1f}% boosting)")
print(f"   • Load variation: {(1 - Q_to_chiller_2/CHILLER_MW_COOLING)*100:.1f}% reduction possible")

print("="*70)


🧮 CALCULATING DYNAMIC CHILLER REQUIREMENTS

📊 Thermal Analysis:
   • Max electrolyser waste heat: 276.0 MW @ 60°C
   • Total booster DH capacity: 110.0 MW_heat
   • 60°C heat per MW DH boost: 0.400 MW
   • Max 60°C to boosters: 44.0 MW

🔍 Scenario 1: Zero Boosting (All Waste Heat to Chiller)
   • All waste heat to chiller: 276.0 MW @ 60°C
   • Chiller thermal load: 276.0 MW

🔍 Scenario 2: Maximum Boosting (High Heat Demand)
   • 60°C to boosters: 44.0 MW
   • 60°C to chiller: 232.0 MW
   • Chiller thermal load: 232.0 MW

🔍 Scenario 3: Minimum Electrolyser Load (No Boosting)
   • Electrolyser at 5% load
   • Waste heat: 13.8 MW @ 60°C
   • Chiller thermal load: 13.8 MW

🎯 CHILLER SIZING:
   • Required cooling capacity: 276.0 MW_thermal
   • COP: 3.0
   • Electrical capacity: 92.0 MW_elec
   • Operating range: 0.5 - 92.0 MW_elec
   • Min load constraint: 10% of capacity

✅ VALIDATION:
   ✅ Can handle 100% electrolyser load with zero boosting
   ✅ Can handle minimum load (232.0 MW) with 

In [24]:
# ==============================
# ==============================
# 🌡️ DYNAMIC CHILLER SYSTEM WITH SINGLE WASTE HEAT BUS
# ==============================
# This implementation uses PyPSA's energy balance with a single waste_heat_bus:
#
# PHYSICAL FLOW:
# 1. Electrolyser outputs waste heat → waste_heat_bus (276 MW at 60°C)
# 2. Boosters extract heat from waste_heat_bus for DH heating
# 3. Bypass allows unused heat to flow through
# 4. Chiller removes remaining heat from waste_heat_bus
#
# KEY PRINCIPLE:
# - PyPSA ensures energy balance: Q_electro = Q_boosters + Q_bypass + Q_chiller
# - More boosting → Less chiller duty → Less electricity consumption
# - Pyomo constraints will enforce: Chiller_elec = Q_chiller / COP
# ==============================

# This implementation properly models the thermal-hydraulic system:
#
# PHYSICAL FLOW PATH:
# 1. Electrolyser heats water: 30°C → 60°C (adds 276 MW)
# 2. Hot water enters waste_heat_bus at 60°C
# 3. Flow splits:
#    a) To boosters: Extracts heat for DH → water cools (e.g., 60°C → 50°C)
#    b) To bypass: No heat extraction → stays at 60°C
# 4. Streams mix: T_mix = f(heat extracted, mass flows)
# 5. Chiller cools: T_mix → 30°C (consumes electricity based on actual ΔT)
#
# KEY PHYSICS:
# - Mass flow proportional to heat carried (at same temperature)
# - Booster extraction: Q_extracted = m × cp × (T_in - T_out)
# - Mixing: T_mix = (Σ m_i × T_i) / Σ m_i
# - Chiller duty: Q_chiller = m_total × cp × (T_mix - 30)
#
# AUTOMATIC CALCULATION IN PyPSA:
# - PyPSA energy balance ensures: Q_electro = Q_to_boosters + Q_to_bypass
# - The chiller on waste_heat_bus will automatically consume the NET heat
# - More boosting → Less heat on bus → Less chiller duty → Less electricity!
# ==============================

print("\n🌡️  CORRECTED: Creating Chiller with Proper Mixing Physics...")
print("="*70)

# ================================================================
# STEP 1: Define thermal parameters
# ================================================================
T_ELECTRO_OUT = 60.0          # °C - Electrolyser outlet temperature
T_ELECTRO_IN = 30.0           # °C - Required chiller outlet (electrolyser inlet)
T_BOOSTER_OUT = 50.0          # °C - Approximate booster HX outlet (cooled water)
CP_WATER = 4.186              # kJ/(kg·K) - Specific heat of water

# ================================================================
# STEP 2: Calculate mass flows and mixing scenarios
# ================================================================
Q_ELECTRO_MAX = ELECTRO_P_NOM * ELECTRO_EFF_HEAT  # 276 MW at 60°C
DELTA_T_ELECTRO = T_ELECTRO_OUT - T_ELECTRO_IN     # 30 K
MASS_FLOW_TOTAL = Q_ELECTRO_MAX / (CP_WATER * DELTA_T_ELECTRO / 1000)  # kg/s

print(f"\n📊 Thermal-Hydraulic Parameters:")
print(f"   • Total heat from electrolyser: {Q_ELECTRO_MAX:.1f} MW")
print(f"   • Temperature rise across electrolyser: {DELTA_T_ELECTRO:.0f} K (30°C → 60°C)")
print(f"   • Total mass flow: {MASS_FLOW_TOTAL:.1f} kg/s")
print(f"   • Booster outlet temperature (approx): {T_BOOSTER_OUT:.0f}°C")

# ================================================================
# STEP 3: Calculate mixing temperatures for different scenarios
# ================================================================
print(f"\n🔍 Mixing Temperature Analysis:")
print(f"   {'Scenario':<30} {'Q_boost':<10} {'Q_bypass':<10} {'T_mix':<8} {'Q_chiller':<12}")
print(f"   {'-'*30} {'-'*10} {'-'*10} {'-'*8} {'-'*12}")

scenarios = []

# Scenario 1: No boosting (worst case for chiller)
Q_boost_1 = 0.0
Q_bypass_1 = Q_ELECTRO_MAX
# All flow bypasses at 60°C
T_mix_1 = T_ELECTRO_OUT
Q_chiller_1 = MASS_FLOW_TOTAL * CP_WATER * (T_mix_1 - T_ELECTRO_IN) / 1000
scenarios.append(("No boosting (0%)", Q_boost_1, Q_bypass_1, T_mix_1, Q_chiller_1))

# Scenario 2: 20% waste heat to boosters
MAX_60C_TO_BOOSTERS = sum(BOOST_CAPACITIES.values()) * FRACTION_FROM_WASTE / HX_EFFECTIVENESS
Q_boost_2 = MAX_60C_TO_BOOSTERS * 0.20  # 20% of max booster capacity
Q_to_booster_hx = Q_boost_2
# Mass flow through boosters
m_booster_2 = Q_to_booster_hx / (CP_WATER * (T_ELECTRO_OUT - T_BOOSTER_OUT) / 1000)
m_bypass_2 = MASS_FLOW_TOTAL - m_booster_2
T_mix_2 = (m_booster_2 * T_BOOSTER_OUT + m_bypass_2 * T_ELECTRO_OUT) / MASS_FLOW_TOTAL
Q_chiller_2 = MASS_FLOW_TOTAL * CP_WATER * (T_mix_2 - T_ELECTRO_IN) / 1000
scenarios.append((f"20% boosting", Q_boost_2, Q_ELECTRO_MAX - Q_boost_2, T_mix_2, Q_chiller_2))

# Scenario 3: 40% waste heat to boosters (high DH demand)
Q_boost_3 = MAX_60C_TO_BOOSTERS * 0.40
m_booster_3 = Q_boost_3 / (CP_WATER * (T_ELECTRO_OUT - T_BOOSTER_OUT) / 1000)
m_bypass_3 = MASS_FLOW_TOTAL - m_booster_3
T_mix_3 = (m_booster_3 * T_BOOSTER_OUT + m_bypass_3 * T_ELECTRO_OUT) / MASS_FLOW_TOTAL
Q_chiller_3 = MASS_FLOW_TOTAL * CP_WATER * (T_mix_3 - T_ELECTRO_IN) / 1000
scenarios.append((f"40% boosting", Q_boost_3, Q_ELECTRO_MAX - Q_boost_3, T_mix_3, Q_chiller_3))

# Scenario 4: Maximum boosting (all booster capacity)
Q_boost_4 = min(MAX_60C_TO_BOOSTERS, Q_ELECTRO_MAX)
m_booster_4 = Q_boost_4 / (CP_WATER * (T_ELECTRO_OUT - T_BOOSTER_OUT) / 1000)
m_bypass_4 = MASS_FLOW_TOTAL - m_booster_4
T_mix_4 = (m_booster_4 * T_BOOSTER_OUT + m_bypass_4 * T_ELECTRO_OUT) / MASS_FLOW_TOTAL
Q_chiller_4 = MASS_FLOW_TOTAL * CP_WATER * (T_mix_4 - T_ELECTRO_IN) / 1000
scenarios.append((f"Max boosting ({Q_boost_4/Q_ELECTRO_MAX*100:.0f}%)", Q_boost_4, Q_ELECTRO_MAX - Q_boost_4, T_mix_4, Q_chiller_4))

for name, q_b, q_bp, t_m, q_c in scenarios:
    print(f"   {name:<30} {q_b:>8.1f} MW {q_bp:>8.1f} MW {t_m:>6.1f}°C {q_c:>10.1f} MW")

# ================================================================
# STEP 4: Key insights from mixing analysis
# ================================================================
print(f"\n💡 Key Insights:")
print(f"   1. Mixing reduces temperature seen by chiller")
print(f"   2. More boosting → Lower T_mix → Less chiller duty")
print(f"   3. Chiller duty range: {Q_chiller_4:.1f} - {Q_chiller_1:.1f} MW")
print(f"   4. Potential chiller reduction: {(Q_chiller_1 - Q_chiller_4)/Q_chiller_1*100:.1f}%")
print(f"   5. This creates STRONG economic incentive for boosting!")

# ================================================================
# STEP 5: Size chiller for worst-case (no boosting)
# ================================================================
CHILLER_MW_COOLING = Q_chiller_1  # Must handle full load at 60°C
CHILLER_MW_ELEC = CHILLER_MW_COOLING / CHILLER_COP

print(f"\n🎯 Chiller Sizing (Worst-Case):")
print(f"   • Required cooling capacity: {CHILLER_MW_COOLING:.1f} MW_thermal")
print(f"   • Input temperature (max): {T_ELECTRO_OUT:.0f}°C")
print(f"   • Output temperature (required): {T_ELECTRO_IN:.0f}°C")
print(f"   • Temperature lift: {T_ELECTRO_OUT - T_ELECTRO_IN:.0f} K")
print(f"   • COP: {CHILLER_COP:.1f}")
print(f"   • Electrical capacity: {CHILLER_MW_ELEC:.1f} MW_elec")

# ================================================================
# STEP 6: Create chiller with CORRECTED setup
# ================================================================
# IMPORTANT: PyPSA automatically handles the energy balance on waste_heat_bus
# The chiller will only consume the NET heat remaining after boosters extract theirs
# This is exactly what we want - the mixing happens implicitly!

if "chiller" in n.links.index:
    n.links.drop("chiller", inplace=True)

n.add(
    "Link",
    "chiller",
    bus0="waste_heat_bus",       # INPUT: Heat removal from 60°C circuit
    bus1="el_bus",                 # OUTPUT: Electricity consumption (negative efficiency)
    efficiency=-1/CHILLER_COP,     # -0.333: For each MW_thermal, consume 0.333 MW_elec
    p_nom=CHILLER_MW_COOLING,      # Maximum cooling capacity (MW_thermal)
    p_min_pu=CHILLER_P_MIN_PU,     # Minimum operating load
    marginal_cost=CHILLER_OPEX_BASE,  # Base OPEX (electricity cost added below)
    committable=True,
    start_up_cost=CHILLER_STARTUP,
    carrier="cooling"
)

# Apply TIME-VARYING electricity cost
# Total cost = (Electricity per thermal × Spot price) + OPEX
chiller_elec_cost_t = CHILLER_ELEC_PER_THERMAL * electricity_price
chiller_total_cost_t = chiller_elec_cost_t + CHILLER_OPEX_BASE
n.links_t.marginal_cost["chiller"] = chiller_total_cost_t.values

print(f"\n✅ Chiller Created with Correct Physics:")
print(f"   • Configuration: bus0=waste_heat_bus, bus1=el_bus")
print(f"   • Cooling capacity: {CHILLER_MW_COOLING:.1f} MW_thermal")
print(f"   • Electrical draw: {CHILLER_MW_ELEC:.1f} MW @ full load")
print(f"   • Efficiency: {-1/CHILLER_COP:.3f} (negative = consumption)")
print(f"   • Min load: {CHILLER_P_MIN_PU*100:.0f}% ({CHILLER_MW_COOLING*CHILLER_P_MIN_PU:.1f} MW)")
print(f"   • Time-varying cost: {chiller_total_cost_t.min():.1f} - {chiller_total_cost_t.max():.1f} DKK/MWh_thermal")

print(f"\n🔗 How PyPSA Automatically Handles Mixing:")
print(f"   1. Energy balance on waste_heat_bus:")
print(f"      Q_from_electrolyser = Q_to_boosters + Q_to_bypass + Q_to_chiller")
print(f"   ")
print(f"   2. When boosters extract 56 MW:")
print(f"      276 MW = 56 MW (boosters) + 0 MW (bypass) + 220 MW (chiller)")
print(f"      → Chiller only removes 220 MW, not 276 MW!")
print(f"   ")
print(f"   3. Chiller electricity = Q_chiller / COP")
print(f"      = 220 MW / 3.0 = 73.3 MW_elec")
print(f"   ")
print(f"   4. More boosting → Less Q_chiller → Less electricity → Lower cost!")

print(f"\n✅ Economic Incentive Structure:")
print(f"   • Scenario 1 (0% boost): Chiller uses {Q_chiller_1/CHILLER_COP:.1f} MW_elec")
print(f"   • Scenario 4 ({Q_boost_4/Q_ELECTRO_MAX*100:.0f}% boost): Chiller uses {Q_chiller_4/CHILLER_COP:.1f} MW_elec")
print(f"   • Electricity savings: {(Q_chiller_1 - Q_chiller_4)/CHILLER_COP:.1f} MW_elec")
print(f"   • At 500 DKK/MWh: {(Q_chiller_1 - Q_chiller_4)/CHILLER_COP * 500:.0f} DKK/hour saved!")
print(f"   • This creates strong preference for waste heat utilization ✅")

print("="*70)


🌡️  CORRECTED: Creating Chiller with Proper Mixing Physics...

📊 Thermal-Hydraulic Parameters:
   • Total heat from electrolyser: 276.0 MW
   • Temperature rise across electrolyser: 30 K (30°C → 60°C)
   • Total mass flow: 2197.8 kg/s
   • Booster outlet temperature (approx): 50°C

🔍 Mixing Temperature Analysis:
   Scenario                       Q_boost    Q_bypass   T_mix    Q_chiller   
   ------------------------------ ---------- ---------- -------- ------------
   No boosting (0%)                    0.0 MW    276.0 MW   60.0°C      276.0 MW
   20% boosting                        8.8 MW    267.2 MW   59.0°C      267.2 MW
   40% boosting                       17.6 MW    258.4 MW   58.1°C      258.4 MW
   Max boosting (16%)                 44.0 MW    232.0 MW   55.2°C      232.0 MW

💡 Key Insights:
   1. Mixing reduces temperature seen by chiller
   2. More boosting → Lower T_mix → Less chiller duty
   3. Chiller duty range: 232.0 - 276.0 MW
   4. Potential chiller reduction: 15.9%
 

In [25]:
# ==============================
# 🔒 CONSTRAINTS - Electrolyser + Chiller System
# ==============================

print("\n🔒 Adding Electrolyser System Constraints...")
print("="*70)

# Fill NaN efficiencies
for col in ['efficiency2', 'efficiency3', 'efficiency4']:
    if col in n.links.columns:
        n.links[col] = n.links[col].fillna(0.0)

# ✅ RECREATE THE MODEL (includes new links)
print("\n🔨 Recreating optimization model (includes new electrolyser components)...")
m = n.optimize.create_model()
p = m.variables["Link-p"]

print("   ✅ Model recreated with all components")


🔒 Adding Electrolyser System Constraints...

🔨 Recreating optimization model (includes new electrolyser components)...


Index(['heat_sink'], dtype='object', name='name')


   ✅ Model recreated with all components


In [26]:
# ==============================
# CHP Constraints - Big-M with Lower Electricity Coupling
# ==============================

import time
start_time = time.time()

snapshots = n.snapshots
heat_dem = n.loads_t.p_set["heat_demand"].reindex(snapshots).fillna(0)

yr = snapshots[0].year
maint_start = pd.Timestamp(f"{yr}-06-01 00:00")
maint_end = pd.Timestamp(f"{yr}-06-20 23:00")
maintenance_hours = snapshots[(snapshots >= maint_start) & (snapshots <= maint_end)]
active_hours = snapshots.difference(maintenance_hours)

print("⚙️  CHP Constraints - Big-M with 0.5× Electricity Coupling...")
print("="*70)

# ================================================================
# Parameters
# ================================================================
HEAT_CAP = CHP_P_HEAT_MAX      # 87.77 MW (TOTAL heat capacity)
FUEL_CAP_TOTAL = CHP_P_ELEC_MAX / CHP_EFF_E         # MW

# ✅ NEW: Split CHP heat capacity between direct and boost
CHP_BOOST_HEAT = 20.0          # MW for boosting (from BOOST_CAPACITIES)
CHP_DIRECT_HEAT = HEAT_CAP - CHP_BOOST_HEAT  # 65.43 MW for direct heating

# Lower electricity coupling to make system feasible
ELEC_HEAT_FUEL_RATIO = 0.08   # 10% instead of 90%

print(f"   CHP Parameters:")
print(f"   • Total heat capacity: {HEAT_CAP:.1f} MW")
print(f"      ├─ Direct heating: {CHP_DIRECT_HEAT:.2f} MW (base load)")
print(f"      └─ Boosting: {CHP_BOOST_HEAT:.1f} MW (with waste heat)")
print(f"   • Elec/Heat fuel ratio: {ELEC_HEAT_FUEL_RATIO:.2f} (reduced for feasibility)")
print(f"   • Total fuel cap: {FUEL_CAP_TOTAL:.1f} MW")

# ================================================================
# 0️⃣ Binary Variable - CHP On/Off Status
# ================================================================
# Check if variable exists from previous run
if 'chp_status' in m.variables.labels:
    print("   ℹ️  Using existing chp_status variable from previous run")
    chp_status = m.variables['chp_status']
else:
    chp_status = m.add_variables(
        coords=[snapshots],
        name="chp_status",
        binary=True
    )
    print(f"   ✅ Binary variable created: chp_status")

# Big-M values
M_HEAT = 200.0
M_FUEL = 400.0

# ================================================================
# 1️⃣ Force CHP ALWAYS ON (except maintenance)
# ================================================================
if len(active_hours) > 0:
    m.add_constraints(
        chp_status.loc[active_hours] == 1,
        name="chp_always_on"
    )
    print(f"   ✅ CHP forced ON during active hours: {len(active_hours):,}")

# ================================================================
# 2️⃣ HEAT CONSTRAINTS (Accounting for Boost Pathway)
# ================================================================
# ✅ KEY INSIGHT: CHP has TWO heat pathways:
#    1. Direct heating (chp_heat): Base load, always running
#    2. Boosting (chp_heat_boost): Uses waste heat, opportunistic
# 
# The direct pathway should operate at (HEAT_CAP - BOOST_CAP) to leave
# room for the boost pathway to utilize waste heat.

low_demand_mask = heat_dem < CHP_DIRECT_HEAT  # Compare to direct capacity only
high_demand_mask = heat_dem >= CHP_DIRECT_HEAT

low_demand_hours = active_hours[low_demand_mask.loc[active_hours]]
high_demand_hours = active_hours[high_demand_mask.loc[active_hours]]

print(f"\n   Hour breakdown (based on direct capacity {CHP_DIRECT_HEAT:.2f} MW):")
print(f"   • Low demand hours: {len(low_demand_hours):,}")
print(f"   • High demand hours: {len(high_demand_hours):,}")

# 2a) LOW DEMAND: CHP direct meets demand (boost is additional)
if len(low_demand_hours) > 0:
    lower_bound_low = heat_dem.loc[low_demand_hours]
    
    # Minimum: Heat ≥ demand × status (for direct pathway)
    m.add_constraints(
        p.loc[low_demand_hours, "chp_heat"] * CHP_EFF_H >= 
        lower_bound_low * chp_status.loc[low_demand_hours],
        name="chp_heat_low_min_100pct"
    )
    
    # Maximum: Allow flexibility up to direct capacity
    m.add_constraints(
        p.loc[low_demand_hours, "chp_heat"] * CHP_EFF_H <= 
        CHP_DIRECT_HEAT * 1.01 * chp_status.loc[low_demand_hours],
        name="chp_heat_low_max"
    )
    
    print(f"\n   ✅ Low demand: CHP direct ≥ demand")
    print(f"      • CHP direct: meets base demand")
    print(f"      • CHP boost: additional {CHP_BOOST_HEAT:.1f} MW when waste heat available")

# 2b) HIGH DEMAND: CHP direct at maximum capacity (65.43 MW)
if len(high_demand_hours) > 0:
    # Direct heating at its full capacity
    m.add_constraints(
        p.loc[high_demand_hours, "chp_heat"] * CHP_EFF_H >= 
        CHP_DIRECT_HEAT * 0.99 * chp_status.loc[high_demand_hours],
        name="chp_heat_high_min"
    )
    
    # Maximum: at direct capacity limit
    m.add_constraints(
        p.loc[high_demand_hours, "chp_heat"] * CHP_EFF_H <= 
        CHP_DIRECT_HEAT * 1.01 * chp_status.loc[high_demand_hours],
        name="chp_heat_high_max"
    )
    
    print(f"   ✅ High demand: CHP direct = {CHP_DIRECT_HEAT:.2f} MW")
    print(f"      • CHP boost: up to {CHP_BOOST_HEAT:.1f} MW additional (when waste heat available)")
    print(f"      • Total CHP contribution: up to {HEAT_CAP:.2f} MW")

# ================================================================
# 3️⃣ ELECTRICITY COUPLING 
# ================================================================
heat_fuel = p.loc[:, "chp_heat"]
elec_fuel = p.loc[:, "chp_el"]

# When ON (status=1): elec_fuel ≥ 0.5 × heat_fuel
# When OFF (status=0): constraint becomes non-binding
m.add_constraints(
    elec_fuel >= ELEC_HEAT_FUEL_RATIO * heat_fuel - M_FUEL * (1 - chp_status),
    name="chp_extraction_coupling"
)

# When OFF: elec_fuel must be 0
m.add_constraints(
    elec_fuel <= M_FUEL * chp_status,
    name="chp_elec_upper"
)

print(f"\n   ✅ Electricity coupling: ≥ {ELEC_HEAT_FUEL_RATIO:.0%} of heat fuel")
print(f"      • Reduced from 90% to 50% for feasibility")
print(f"      • Still ensures proper electricity generation")

# ================================================================
# 4️⃣ CAPACITY LIMITS
# ================================================================
# ✅ Direct heat capacity (reduced by boost allocation)
m.add_constraints(
    p.loc[:, "chp_heat"] * CHP_EFF_H <= CHP_DIRECT_HEAT,
    name="chp_heat_capacity"
)

# Electricity capacity (unchanged)
m.add_constraints(
    p.loc[:, "chp_el"] * CHP_EFF_E <= CHP_P_ELEC_MAX,
    name="chp_elec_capacity"
)

# Total fuel cap (shared between direct + boost, handled in shared capacity constraint)
m.add_constraints(
    p.loc[:, "chp_el"] + p.loc[:, "chp_heat"] <= FUEL_CAP_TOTAL,
    name="chp_fuel_cap"
)

print(f"   ✅ Capacity limits:")
print(f"      • Direct heat: ≤ {CHP_DIRECT_HEAT:.2f} MW")
print(f"      • Boost heat: ≤ {CHP_BOOST_HEAT:.1f} MW (separate link)")
print(f"      • Elec: ≤ {CHP_P_ELEC_MAX:.1f} MW")
print(f"      • Fuel: ≤ {FUEL_CAP_TOTAL:.1f} MW (shared with boost)")
print(f"\n   ℹ️  Note: chp_heat + chp_heat_boost fuel inputs are constrained")
print(f"      by shared fuel capacity in Section 3 of coupling constraints")

# ================================================================
# 5️⃣ MAINTENANCE: Force OFF (including chp_heat_boost)
# ================================================================
if len(maintenance_hours):
    m.add_constraints(
        chp_status.loc[maintenance_hours] == 0,
        name="chp_maintenance_off"
    )
    m.add_constraints(p.loc[maintenance_hours, "chp_heat"] == 0)
    m.add_constraints(p.loc[maintenance_hours, "chp_el"] == 0)
    
    # ✅ NEW: Also turn OFF chp_heat_boost during maintenance
    # Since chp_heat_boost is part of the CHP system (uses 20 MW of total 85.43 MW capacity)
    if "chp_heat_boost" in n.links.index:
        m.add_constraints(p.loc[maintenance_hours, "chp_heat_boost"] == 0)
        print(f"\n   ✅ Maintenance: CHP and CHP Boost OFF June 1-20 ({len(maintenance_hours)} hours)")
        print(f"      • chp_heat: OFF")
        print(f"      • chp_el: OFF")
        print(f"      • chp_heat_boost: OFF (part of CHP system)")
    else:
        print(f"\n   ✅ Maintenance: CHP OFF June 1-20 ({len(maintenance_hours)} hours)")

# ================================================================
# 6️⃣ CHP BASE LOAD WHEN ELECTROLYSER IS OFF
# ================================================================
print(f"\n   🔧 CHP Base Load Priority (Electrolyser Coupling):")

# Get electrolyser status variable if it exists
if 'Link-status' in m.variables.labels:
    link_status = m.variables['Link-status']
    
    if 'electrolyser' in link_status.coords['name'].values:
        s_electro = link_status.loc[:, 'electrolyser']
        
        # Constraint: When electrolyser is OFF, CHP must use direct heating for base load
        # 
        # Logic: chp_heat (fuel) >= CHP_BASE_FUEL × chp_status × (1 - s_electro)
        # 
        # Case 1: Electrolyser OFF (s_electro=0), CHP ON (chp_status=1)
        #         → chp_heat >= CHP_BASE_FUEL (ENFORCED - must use direct heating)
        # 
        # Case 2: Electrolyser ON (s_electro=1), CHP ON (chp_status=1)  
        #         → chp_heat >= 0 (RELAXED - can use boosting instead)
        # 
        # Case 3: CHP OFF (chp_status=0)
        #         → chp_heat >= 0 (RELAXED)
        
        # Use the full direct heating capacity as base load requirement
        CHP_BASE_HEAT = CHP_DIRECT_HEAT  # 65.43 MW heat output
        CHP_BASE_FUEL = CHP_BASE_HEAT / CHP_EFF_H  # 109.05 MW fuel input
        
        # ✅ FIXED: Use vectorized operations WITHOUT .loc[active_hours] 
        # to avoid coordinate mismatch issues
        #
        # The constraint applies to ALL timesteps:
        # - During maintenance: chp_status=0, so constraint becomes 0 ≥ 0 (satisfied)
        # - During active hours: constraint enforces base load when electro OFF
        
        # Problem: chp_status * (1 - s_electro) creates QUADRATIC constraint (binary × binary)
        # Solution: Linearize using auxiliary binary variable
        #
        # Define: chp_and_no_electro = 1 IFF (chp_status=1 AND s_electro=0)
        # Linearization (standard Big-M technique):
        #   chp_and_no_electro <= chp_status                    [can't be 1 if CHP OFF]
        #   chp_and_no_electro <= (1 - s_electro)               [can't be 1 if electrolyser ON]
        #   chp_and_no_electro >= chp_status + (1-s_electro) - 1 [must be 1 if both true]
        
        chp_and_no_electro = m.add_variables(
            coords=[snapshots],
            name="chp_and_no_electro",
            binary=True
        )
        
        # Linearization constraints
        m.add_constraints(
            chp_and_no_electro - chp_status <= 0,
            name="aux1_chp_and_no_electro"
        )
        m.add_constraints(
            chp_and_no_electro + s_electro <= 1,
            name="aux2_chp_and_no_electro"
        )
        m.add_constraints(
            chp_and_no_electro - chp_status + s_electro >= 0,
            name="aux3_chp_and_no_electro"
        )
        
        # Main constraint using linearized variable
        constraint_expr = p.loc[:, 'chp_heat'] - CHP_BASE_FUEL * chp_and_no_electro
        
        m.add_constraints(
            constraint_expr >= 0,
            name="chp_direct_when_electro_off"
        )
        
        print(f"      ✅ Constraint added: ALL timesteps ({len(snapshots):,})")
        print(f"      • CHP base load: {CHP_BASE_HEAT:.2f} MW heat ({CHP_BASE_FUEL:.2f} MW fuel)")
        print(f"      • When electrolyser OFF: CHP uses direct heating (prevents HP infiltration)")
        print(f"      • When electrolyser ON: CHP can use boosting (constraint relaxed)")
        print(f"      • Uses linearized auxiliary variable (avoids quadratic expression)")
        
    else:
        print(f"      ⚠️  Electrolyser status not found, constraint skipped")
else:
    print(f"      ⚠️  Link-status variable not found, constraint skipped")

print(f"\n⏱️  CHP constraints built in {time.time() - start_time:.1f} seconds")

⚙️  CHP Constraints - Big-M with 0.5× Electricity Coupling...
   CHP Parameters:
   • Total heat capacity: 85.4 MW
      ├─ Direct heating: 65.43 MW (base load)
      └─ Boosting: 20.0 MW (with waste heat)
   • Elec/Heat fuel ratio: 0.08 (reduced for feasibility)
   • Total fuel cap: 216.7 MW
   ✅ Binary variable created: chp_status
   ✅ CHP forced ON during active hours: 8,280

   Hour breakdown (based on direct capacity 65.43 MW):
   • Low demand hours: 1,616
   • High demand hours: 6,664

   ✅ Low demand: CHP direct ≥ demand
      • CHP direct: meets base demand
      • CHP boost: additional 20.0 MW when waste heat available
   ✅ High demand: CHP direct = 65.43 MW
      • CHP boost: up to 20.0 MW additional (when waste heat available)
      • Total CHP contribution: up to 85.43 MW

   ✅ Electricity coupling: ≥ 8% of heat fuel
      • Reduced from 90% to 50% for feasibility
      • Still ensures proper electricity generation


c:\Users\elshq\myenv\Lib\site-packages\linopy\common.py:437: UserWarning:

Coordinates across variables not equal. Perform outer join.



   ✅ Capacity limits:
      • Direct heat: ≤ 65.43 MW
      • Boost heat: ≤ 20.0 MW (separate link)
      • Elec: ≤ 65.0 MW
      • Fuel: ≤ 216.7 MW (shared with boost)

   ℹ️  Note: chp_heat + chp_heat_boost fuel inputs are constrained
      by shared fuel capacity in Section 3 of coupling constraints

   ✅ Maintenance: CHP OFF June 1-20 (480 hours)

   🔧 CHP Base Load Priority (Electrolyser Coupling):
      ✅ Constraint added: ALL timesteps (8,760)
      • CHP base load: 65.43 MW heat (109.05 MW fuel)
      • When electrolyser OFF: CHP uses direct heating (prevents HP infiltration)
      • When electrolyser ON: CHP can use boosting (constraint relaxed)
      • Uses linearized auxiliary variable (avoids quadratic expression)

⏱️  CHP constraints built in 0.5 seconds


c:\Users\elshq\myenv\Lib\site-packages\linopy\common.py:437: UserWarning:

Coordinates across variables not equal. Perform outer join.



In [27]:
# ===============================================================# ✅ ELECTROLYSER INTEGRATION - COUPLING CONSTRAINTS# ===============================================================print("\n" + "="*70)print("🔧 ELECTROLYSER SYSTEM - COUPLING CONSTRAINTS")print("="*70)# ================================================================# 1️⃣ CHILLER - AUTOMATIC VIA ENERGY BALANCE (No explicit constraints needed)# ================================================================print(f"\n1️⃣ Chiller Operation:")print(f"   ✅ Chiller runs automatically via waste_heat_bus energy balance")print(f"   💡 No sync constraints needed:")print(f"      • PyPSA energy balance: Electrolyser_out = Boosters + Chiller + Bypass")print(f"      • Chiller consumes residual waste heat after boosters extract what they need")print(f"      • This allows boosters to maximize waste heat utilization!")print(f"   ⚠️  Previous sync constraints (s_chiller = s_electro) were REMOVED")print(f"      • They forced chiller to always run, blocking booster access to waste heat")print(f"      • Result: Booster usage was only 0.73% instead of ~70%")print("="*70)# ================================================================# 2️⃣ BOOSTER - ELECTROLYSER COUPLING (Big-M method)# ================================================================print(f"\n2️⃣ Booster-Electrolyser Coupling:")if 'Link-status' in m.variables.labels:    status = m.variables['Link-status']        if 'electrolyser' in status.coords['name'].values:        s_electro = status.loc[:, 'electrolyser']                print(f"\n   🔗 Adding booster-electrolyser coupling constraints...")        print(f"   💡 Purpose: Boosters can only run when electrolyser provides waste heat")                # Use a list to track processed items to guarantee order and uniqueness        processed_list = []                for component, boost_heat_mw in BOOST_CAPACITIES.items():            boost_link = f"{component}_boost"                        # Skip if already processed OR if link doesn't exist            if boost_link in processed_list:                continue                            if boost_link not in n.links.index:                continue                        # Check if booster has power variable (skip if not)            if boost_link not in p.coords['name'].values:                print(f"      ⚠️  {boost_link}: No power variable found, skipping")                continue                        # Add to processed list            processed_list.append(boost_link)                        # Get booster fuel capacity            boost_fuel_cap = n.links.at[boost_link, 'p_nom']            M_BOOST = boost_fuel_cap * 1.1                        # Get the power variable for this booster            p_boost = p.loc[:, boost_link]                        # Create the Big-M constraint for each timestep            # p_boost(t) <= M_BOOST * s_electro(t)            for t in n.snapshots:                m.add_constraints(                    p_boost.loc[t] <= M_BOOST * s_electro.loc[t],                    name=f"{boost_link}_electro_{t.strftime('%Y%m%d%H%M')}"                )                        print(f"      ✅ {boost_link}: {len(n.snapshots)} constraints")                print(f"\n   ✅ Booster coupling complete ({len(processed_list)} boosters)")    else:        print(f"   ⚠️  Electrolyser status variable not found")else:    print(f"   ⚠️  Link-status not in model variables")print("="*70)# ================================================================# 3️⃣ SHARED BOILER CAPACITY# ================================================================print(f"\n3️⃣ Shared Boiler Fuel Input Capacity:")print(f"   💡 Purpose: Direct + Boost modes share same physical fuel input")# Use list to track processed componentsprocessed_list = []for component, boost_heat_mw in BOOST_CAPACITIES.items():    boost_link = f"{component}_boost"        # Skip if already processed or if component/booster doesn't exist    if component in processed_list:        continue        if component not in n.links.index or boost_link not in n.links.index:        continue        # Add to processed list    processed_list.append(component)        fuel_cap = n.links.at[component, 'p_nom']        constraint_expr = p.loc[:, component] + p.loc[:, boost_link]        m.add_constraints(        constraint_expr <= fuel_cap,        name=f"{component}_shared_fuel"    )        print(f"   ✅ {component}: {len(n.snapshots)} constraints")    print(f"      • Fuel capacity: {fuel_cap:.1f} MW")print("="*70)# ================================================================# 🔍 SUMMARY OF CONSTRAINTS# ================================================================print(f"\n🔍 CONSTRAINT SUMMARY:")print(f"   ✅ Booster-Electrolyser coupling: Enforces boosters only run with waste heat")print(f"   ✅ Shared fuel capacity: Prevents Direct + Boost from exceeding boiler capacity")print(f"   ❌ Chiller sync: REMOVED - Was blocking booster waste heat access")print(f"\n   💡 Result: Boosters now free to maximize waste heat utilization!")print("="*70)


🔧 ELECTROLYSER SYSTEM - COUPLING CONSTRAINTS

1️⃣ Chiller-Electrolyser Coupling:

   🔗 Adding chiller-electrolyser coupling (vectorized)...
      ✅ Vectorized constraints added: 8,760 timesteps
         • Constraint 1: s_chiller ≤ s_electro (can't run without electrolyser)
         • Constraint 2: s_chiller ≥ s_electro (must run with electrolyser)
         • Result: Perfect synchronization (s_chiller = s_electro)
      💡 Using vectorized operations like CHP constraints (fast!)

2️⃣ Booster-Electrolyser Coupling:

   🔗 Adding booster-electrolyser coupling constraints...


c:\Users\elshq\myenv\Lib\site-packages\linopy\common.py:437: UserWarning:

Coordinates across variables not equal. Perform outer join.

c:\Users\elshq\myenv\Lib\site-packages\linopy\common.py:437: UserWarning:

Coordinates across variables not equal. Perform outer join.



      ✅ chp_heat_boost: 8760 constraints
      ✅ electric_boiler_boost: 8760 constraints
      ✅ gas_boiler_boost: 8760 constraints
      ✅ biomass_boiler_boost: 8760 constraints

   ✅ Booster coupling complete (4 boosters)

3️⃣ Shared Boiler Fuel Input Capacity:
   ✅ chp_heat: 8760 constraints
      • Fuel capacity: 142.4 MW
   ✅ electric_boiler: 8760 constraints
      • Fuel capacity: 40.4 MW
   ✅ gas_boiler: 8760 constraints
      • Fuel capacity: 75.0 MW
   ✅ biomass_boiler: 8760 constraints
      • Fuel capacity: 76.0 MW

🔍 CONSTRAINT VERIFICATION:
   Checking if constraints are actually duplicated in the model...


c:\Users\elshq\myenv\Lib\site-packages\linopy\common.py:437: UserWarning:

Coordinates across variables not equal. Perform outer join.




   📊 Booster coupling constraints in model:
      • Total booster-electro constraints: 35041
      • Unique booster types: 5 → {'chp_direct_when', 'biomass_boiler_boost', 'chp_heat_boost', 'electric_boiler_boost', 'gas_boiler_boost'}
      • Expected per booster: 8760 (snapshots)
      • Expected total: 43800

   ⚠️  WARNING: Constraint count doesn't match!
      → Actual: 35041, Expected: 43800


In [ ]:
# ===============================================================
# 🔧 DYNAMIC CHILLER LOAD - PYOMO CONSTRAINTS
# ===============================================================
# This implements the dynamic chiller electricity consumption model
# where chiller electricity varies based on actual waste heat utilization.
#
# Model: Chiller_electricity[t] = (Waste_heat[t] - Heat_utilized[t]) / COP
# ===============================================================

print("\n" + "="*70)
print("🔧 DYNAMIC CHILLER LOAD - PYOMO CONSTRAINTS")
print("="*70)

# Get the necessary variables
p = m.variables["Link-p"]

# Get time indices
snapshots = n.snapshots

print(f"\n⚡ Implementing dynamic chiller electricity calculation...")
print(f"   Formula: Chiller_elec[t] = Heat_rejected[t] / COP")
print(f"   Where: Heat_rejected[t] = max(Waste_heat[t] - Heat_to_boosters[t] - Heat_to_bypass[t], 0)")

# Get the chiller thermal capacity (p0 is the heat removed from waste_heat_bus)
if "chiller" in n.links.index:
    chiller_thermal = p.loc[:, "chiller"]
    
    # Chiller electricity consumption (p1 is the electricity draw)
    # The electricity consumption should equal thermal load / COP
    # This is automatically enforced by the efficiency=-1/COP in the Link definition
    
    print(f"\n✅ Chiller electricity consumption enforced by Link efficiency:")
    print(f"   • Efficiency: {n.links.at['chiller', 'efficiency']:.3f}")
    print(f"   • This means: p1 = p0 × efficiency = p0 × (-1/{CHILLER_COP})")
    print(f"   • So: Electricity[t] = Thermal[t] / {CHILLER_COP}")
    
    # The energy balance on waste_heat_bus ensures:
    # Electrolyser output = Booster consumption + Bypass + Chiller removal
    # This is automatically handled by PyPSA
    
    print(f"\n✅ Energy balance on waste_heat_bus (automatic):")
    print(f"   • Electrolyser waste heat output (p2)")
    print(f"   • = Booster heat consumption (p2, negative efficiency)")  
    print(f"   • + Bypass heat (p0)")
    print(f"   • + Chiller heat removal (p0)")
    
    print(f"\n✅ Dynamic chiller load is fully modeled!")
    print(f"   • Chiller heat removal varies automatically with booster usage")
    print(f"   • Chiller electricity = Heat removal / COP (via Link efficiency)")
    print(f"   • No additional Pyomo constraints needed!")
    
else:
    print(f"   ⚠️ Chiller link not found in model")

print("="*70)


In [28]:
# ==============================
# Set Ramp Limits via DataFrame (workaround for PyPSA bug)
# ==============================

print("⚙️  Setting ramp limits...")

# Set ramp limits directly in the links DataFrame
n.links.loc["gas_boiler", "ramp_limit_up"] = GB_RAMP_LIMIT      # 0.20
n.links.loc["gas_boiler", "ramp_limit_down"] = GB_RAMP_LIMIT    # 0.20

n.links.loc["biomass_boiler", "ramp_limit_up"] = BB_RAMP_LIMIT      # 0.20
n.links.loc["biomass_boiler", "ramp_limit_down"] = BB_RAMP_LIMIT    # 0.20

n.links.loc["electrolyser", "ramp_limit_up"] = ELECTRO_RAMP_UP
n.links.loc["electrolyser", "ramp_limit_down"] = ELECTRO_RAMP_DOWN
print(f"   ✅ Ramp limits: ±{ELECTRO_RAMP_UP*100:.0f}%/hour")


# Verify they're set
print(f"✅ Gas boiler ramp limits: ±{n.links.at['gas_boiler', 'ramp_limit_up']:.0%} of p_nom")
print(f"✅ Biomass boiler ramp limits: ±{n.links.at['biomass_boiler', 'ramp_limit_up']:.0%} of p_nom")
print("="*70)

⚙️  Setting ramp limits...
   ✅ Ramp limits: ±75%/hour
✅ Gas boiler ramp limits: ±20% of p_nom
✅ Biomass boiler ramp limits: ±20% of p_nom


In [29]:
# ==============================
# 🚀 SOLVE WITH ELECTROLYSER SYSTEM
# ==============================

print("\n🚀 SOLVING WITH ELECTROLYSER SYSTEM")
print("="*70)

print(f"\n📊 Configuration Summary:")
print(f"   • Electrolyser: {ELECTRO_P_NOM:.0f} MW (break-even: {-electro_marginal_cost:.1f} DKK/MWh)")
# print(f"   • Chiller: {CHILLER_P_NOM:.1f} MW_el ({CHILLER_MW_COOLING:.1f} MW_cooling)")
print(f"   • Boosting: {sum(BOOST_CAPACITIES.values()):.0f} MW_heat capacity")
print(f"   • Bypass: Unlimited (for excess heat)")

profitable_hours = (electricity_price < -electro_marginal_cost).sum()
print(f"   • Profitable hours: {profitable_hours} / 8760 ({profitable_hours/8760*100:.1f}%)")

print(f"\n⏳ Solving (this may take 5-15 minutes)...")

status = n.optimize.solve_model(solver_name='gurobi')

print(f"\n✅ Solve: {status}")
print(f"   Objective: {n.objective:,.0f} DKK")

INFO:linopy.model: Solve problem using Gurobi solver



🚀 SOLVING WITH ELECTROLYSER SYSTEM

📊 Configuration Summary:
   • Electrolyser: 1000 MW (break-even: 627.0 DKK/MWh)
   • Boosting: 110 MW_heat capacity
   • Bypass: Unlimited (for excess heat)
   • Profitable hours: 5627 / 8760 (64.2%)

⏳ Solving (this may take 5-15 minutes)...


INFO:linopy.io:Writing objective.
Writing binary variables.: 100%|██████████| 5/5 [00:00<00:00, 20.00it/s]
INFO:linopy.io: Writing time: 95.41s


Set parameter Username


INFO:gurobipy:Set parameter Username


...

Optimal solution found (tolerance 1.00e-04)


INFO:gurobipy:Optimal solution found (tolerance 1.00e-04)


Best objective -1.105559433971e+09, best bound -1.105668875005e+09, gap 0.0099%


INFO:gurobipy:Best objective -1.105559433971e+09, best bound -1.105668875005e+09, gap 0.0099%
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 376680 primals, 0 duals
Objective: -1.11e+09
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The variable 'chp_status' could not be mapped to the network component because it does not include the symbol '-'.
INFO:pypsa.optimization.optimize:The variable 'chp_and_no_electro' could not be mapped to the network component because it does not include the symbol '-'.
INFO:pypsa.optimization.optimize:No shadow prices were assigned to the network.



✅ Solve: ('ok', 'optimal')
   Objective: -1,105,559,434 DKK


In [30]:
# ================================================================
# 1️⃣ SOLVER STATUS & SYSTEM OVERVIEW
# ================================================================

print("\n" + "="*70)
print("1️⃣ SOLVER STATUS & SYSTEM OVERVIEW")
print("="*70)

# Solver status (different for PyPSA versions)
print(f"\n📋 Solver Information:")
try:
    # Try newer PyPSA API
    if hasattr(n, 'optimization_status'):
        print(f"   • Status: {n. optimization_status.get('status', 'Unknown')}")
        print(f"   • Termination: {n.optimization_status.get('termination_condition', 'Unknown')}")
        solve_time = n.optimization_status.get('time', 0)
    # Try older PyPSA API
    elif hasattr(n, 'results'):
        print(f"   • Status: {n.results. get('Solver', [{}])[0].get('Status', 'Unknown')}")
        print(f"   • Termination: {n.results.get('Solver', [{}])[0].get('Termination condition', 'Unknown')}")
        solve_time = n.results. get('Solver', [{}])[0].get('Time', 0)
    else:
        print(f"   • Status: Solved (details unavailable)")
        solve_time = 0
    
    print(f"   • Solve time: {solve_time:. 1f} seconds")
    
    # Objective value
    if hasattr(n, 'objective'):
        print(f"   • Objective value: {n.objective:,.0f} DKK")
    elif hasattr(m, 'objective'):
        print(f"   • Objective value: {m.objective. value:,.0f} DKK")
    else:
        print(f"   • Objective value: (unavailable)")
        
except Exception as e:
    print(f"   • Status: Solved (assuming success)")
    print(f"   • Error getting details: {e}")

# System totals
print(f"\n⚡ Electricity System:")
try:
    total_elec_demand = n.loads_t.p_set.sum(). sum() if hasattr(n. loads_t. p_set, 'sum') else 0
    print(f"   • Total demand: {total_elec_demand:,.0f} MWh")
except:
    print(f"   • Total demand: (unavailable)")

try:
    total_elec_gen = n.generators_t.p. sum().sum() if hasattr(n.generators_t, 'p') else 0
    print(f"   • Total generation: {total_elec_gen:,.0f} MWh")
except:
    print(f"   • Total generation: (unavailable)")

print(f"\n🔥 Heat System:")
try:
    total_heat_demand = n.loads_t.p_set["heat_demand"].sum()
    print(f"   • Total demand: {total_heat_demand:,.0f} MWh")
    print(f"   • Avg demand: {n.loads_t.p_set['heat_demand'].mean():.1f} MW")
    print(f"   • Peak demand: {n.loads_t.p_set['heat_demand'].max():.1f} MW")
except Exception as e:
    print(f"   • Error getting heat data: {e}")

print("="*70)


1️⃣ SOLVER STATUS & SYSTEM OVERVIEW

📋 Solver Information:
   • Status: Solved (details unavailable)
   • Status: Solved (assuming success)
   • Error getting details: Format specifier missing precision

⚡ Electricity System:
   • Total demand: 1,236,001 MWh
   • Total generation: 7,002,485 MWh

🔥 Heat System:
   • Total demand: 1,236,001 MWh
   • Avg demand: 141.1 MW
   • Peak demand: 330.8 MW


In [31]:
# ================================================================
# 1 SOLVER STATUS & SYSTEM OVERVIEW
# ================================================================

print("\n" + "="*70)
print("1 SOLVER STATUS & SYSTEM OVERVIEW")
print("="*70)

print(f"\n📋 Optimization Status:")
print(f"   • Status: Solved successfully")

print(f"\n⚡ Electricity System:")
total_elec_demand = n. loads_t.p_set.sum().sum()
total_elec_gen = n.generators_t.p.sum().sum()
print(f"   • Total demand: {total_elec_demand:,.0f} MWh")
print(f"   • Total generation: {total_elec_gen:,.0f} MWh")

print(f"\n🔥 Heat System:")
total_heat_demand = n.loads_t.p_set["heat_demand"].sum()
print(f"   • Total demand: {total_heat_demand:,.0f} MWh")
print(f"   • Avg demand: {n.loads_t.p_set['heat_demand'].mean():.1f} MW")
print(f"   • Peak demand: {n.loads_t.p_set['heat_demand'].max():.1f} MW")

print("="*70)


1 SOLVER STATUS & SYSTEM OVERVIEW

📋 Optimization Status:
   • Status: Solved successfully

⚡ Electricity System:
   • Total demand: 1,236,001 MWh
   • Total generation: 7,002,485 MWh

🔥 Heat System:
   • Total demand: 1,236,001 MWh
   • Avg demand: 141.1 MW
   • Peak demand: 330.8 MW


In [32]:
# ================================================================
# 2 ELECTROLYSER PERFORMANCE
# ================================================================

print("\n" + "="*70)
print("2 ELECTROLYSER PERFORMANCE")
print("="*70)

electro_p = n.links_t.p0["electrolyser"]
electro_h2 = n.links_t.p1["electrolyser"]
electro_heat = n.links_t.p2["electrolyser"]. abs()

operating_mask = electro_p > 1
operating_hours = operating_mask.sum()
total_electricity = electro_p.sum()
total_h2 = electro_h2.sum()
total_waste_heat = electro_heat. sum()

print(f"\n📊 Operation Statistics:")
print(f"   • Operating hours: {operating_hours:,} / 8,760 ({operating_hours/8760*100:.1f}%)")
print(f"   • Capacity factor: {(total_electricity/(ELECTRO_P_NOM*8760))*100:.1f}%")
print(f"   • Average load: {electro_p[operating_mask].mean():.1f} MW ({electro_p[operating_mask].mean()/ELECTRO_P_NOM*100:.1f}%)")
print(f"   • Min load (when ON): {electro_p[operating_mask].min():.1f} MW")
print(f"   • Max load: {electro_p. max():.1f} MW")

print(f"\n⚡ Energy Flows:")
print(f"   • Total electricity consumed: {total_electricity:,.0f} MWh")
print(f"   • Total H2 produced: {total_h2:,.0f} MWh_H2")
print(f"   • Total waste heat: {total_waste_heat:,.0f} MWh_thermal")

print(f"\n💰 Economics:")
electro_cost = n.links. at['electrolyser', 'marginal_cost']
h2_revenue_rate = -electro_cost
total_gross_revenue = total_electricity * h2_revenue_rate
electricity_cost = total_electricity * (DSO_TARIFF + OPEX_PER_MWH_ELEC)
net_profit = total_gross_revenue - electricity_cost

print(f"   • Marginal cost: {electro_cost:.1f} DKK/MWh")
print(f"   • H2 revenue: {total_h2 * H2_REVENUE / ELECTRO_EFF_H2:,.0f} DKK")
print(f"   • Electricity cost: {electricity_cost:,.0f} DKK")
print(f"   • Net profit: {net_profit:,.0f} DKK")

if "electrolyser" in n.links_t.status. columns:
    status = n.links_t.status["electrolyser"]
    startups = ((status. diff() > 0.5) & (status > 0.5)).sum()
    startup_cost = startups * ELECTRO_STARTUP
    print(f"\n🔄 Cycling:")
    print(f"   • Number of start-ups: {startups}")
    print(f"   • Total startup cost: {startup_cost:,.0f} DKK")
else:
    startups = 0
    print(f"\n🔄 Cycling:")
    print(f"   • Status data not available")

print("="*70)


2 ELECTROLYSER PERFORMANCE

📊 Operation Statistics:
   • Operating hours: 6,683 / 8,760 (76.3%)
   • Capacity factor: 54.7%
   • Average load: 716.8 MW (71.7%)
   • Min load (when ON): 100.0 MW
   • Max load: 1000.0 MW

⚡ Energy Flows:
   • Total electricity consumed: 4,790,475 MWh
   • Total H2 produced: -2,507,335 MWh_H2
   • Total waste heat: 1,322,171 MWh_thermal

💰 Economics:
   • Marginal cost: -627.0 DKK/MWh
   • H2 revenue: -178,684,730 DKK
   • Electricity cost: 410,992,605 DKK
   • Net profit: 2,592,679,988 DKK

🔄 Cycling:
   • Number of start-ups: 302
   • Total startup cost: 2,252,920 DKK


In [33]:
# ================================================================
# 3 WASTE HEAT UTILIZATION
# ================================================================

print("\n" + "="*70)
print("3 WASTE HEAT UTILIZATION")
print("="*70)

chiller_thermal = n.links_t.p0["chiller"]. abs()
chiller_total = chiller_thermal.sum()

boost_60c_total = 0
boost_breakdown = {}
for boost in ['chp_heat_boost', 'electric_boiler_boost', 'gas_boiler_boost', 'biomass_boiler_boost']:
    if boost in n.links_t.p2. columns:
        boost_cons = n.links_t.p2[boost]. abs()
        boost_total = boost_cons.sum()
        boost_60c_total += boost_total
        boost_breakdown[boost] = boost_total

bypass_total = 0
if "waste_heat_bypass" in n.links_t.p0.columns:
    bypass_total = n.links_t.p0["waste_heat_bypass"].abs().sum()

print(f"\n📊 Waste Heat Balance (Annual):")
print(f"   • Produced by electrolyser: {total_waste_heat:,.0f} MWh")
print(f"   • To boosters: {boost_60c_total:,.0f} MWh ({boost_60c_total/total_waste_heat*100:.1f}%)")
print(f"   • To chiller: {chiller_total:,.0f} MWh ({chiller_total/total_waste_heat*100:.1f}%)")
print(f"   • Bypassed: {bypass_total:,.0f} MWh ({bypass_total/total_waste_heat*100:.1f}%)")
print(f"   • Total out: {boost_60c_total + chiller_total + bypass_total:,.0f} MWh")

balance_error = abs(total_waste_heat - (boost_60c_total + chiller_total + bypass_total))
print(f"   • Balance error: {balance_error:,.0f} MWh ({balance_error/total_waste_heat*100:.2f}%)")

if balance_error/total_waste_heat < 0.01:
    print(f"   ✅ Heat balance verified (< 1% error)")
else:
    print(f"   ⚠️ Heat balance error > 1%")

print(f"\n🔥 Booster Consumption Breakdown:")
for boost, total in sorted(boost_breakdown.items(), key=lambda x: x[1], reverse=True):
    pct = (total / boost_60c_total * 100) if boost_60c_total > 0 else 0
    avg_when_on = total / operating_hours if operating_hours > 0 else 0
    print(f"   • {boost:30s}: {total:>10,.0f} MWh ({pct:>5.1f}%) - Avg: {avg_when_on:>6.2f} MW")

print("="*70)


3 WASTE HEAT UTILIZATION

📊 Waste Heat Balance (Annual):
   • Produced by electrolyser: 1,322,171 MWh
   • To boosters: 142,861 MWh (10.8%)
   • To chiller: 1,179,310 MWh (89.2%)
   • Bypassed: 0 MWh (0.0%)
   • Total out: 1,322,171 MWh
   • Balance error: 0 MWh (0.00%)
   ✅ Heat balance verified (< 1% error)

🔥 Booster Consumption Breakdown:
   • biomass_boiler_boost          :     75,075 MWh ( 52.6%) - Avg:  11.23 MW
   • chp_heat_boost                :     32,999 MWh ( 23.1%) - Avg:   4.94 MW
   • electric_boiler_boost         :     27,961 MWh ( 19.6%) - Avg:   4.18 MW
   • gas_boiler_boost              :      6,825 MWh (  4.8%) - Avg:   1.02 MW


In [34]:
# ================================================================
# 🔍 VERIFY DYNAMIC CHILLER LOAD (Bus Energy Balance Check)
# ================================================================

print("\n" + "="*70)
print("🔍 DYNAMIC CHILLER LOAD VERIFICATION")
print("="*70)

# Get time series data
electro_waste_heat = n.links_t.p2["electrolyser"].abs()  # Heat produced by electrolyser

# Boosters extract heat via bus2 (negative efficiency2)
boost_heat_extracted = 0
for boost in ['chp_heat_boost', 'electric_boiler_boost', 'gas_boiler_boost', 'biomass_boiler_boost']:
    if boost in n.links_t.p2.columns:
        boost_heat_extracted += n.links_t.p2[boost].abs()  # Heat consumed by boosters

# Chiller heat removed (from bus0 - thermal power)
chiller_heat_removed = n.links_t.p0["chiller"].abs()

# Bypass heat (if exists)
bypass_heat = 0
if "waste_heat_bypass" in n.links_t.p0.columns:
    bypass_heat = n.links_t.p0["waste_heat_bypass"].abs()

# Chiller electricity consumption (from bus1 - electrical power)
chiller_elec = n.links_t.p1["chiller"].abs()

print(f"\n📊 Annual Energy Balance (waste_heat_bus):")
print(f"   Heat IN:")
print(f"      • From electrolyser: {electro_waste_heat.sum():,.0f} MWh")
print(f"   ")
print(f"   Heat OUT:")
print(f"      • To boosters (for 40°C→60°C base heating): {boost_heat_extracted.sum():,.0f} MWh ({boost_heat_extracted.sum()/electro_waste_heat.sum()*100:.1f}%)")
print(f"      • To bypass: {bypass_heat.sum():,.0f} MWh ({bypass_heat.sum()/electro_waste_heat.sum()*100:.1f}%)")
print(f"      • To chiller: {chiller_heat_removed.sum():,.0f} MWh ({chiller_heat_removed.sum()/electro_waste_heat.sum()*100:.1f}%)")
print(f"   ")
print(f"   Total OUT: {(boost_heat_extracted + bypass_heat + chiller_heat_removed).sum():,.0f} MWh")

# Energy balance check
balance_error = electro_waste_heat.sum() - (boost_heat_extracted + bypass_heat + chiller_heat_removed).sum()
balance_pct = abs(balance_error / electro_waste_heat.sum() * 100)

if balance_pct < 0.1:
    print(f"   ✅ Balance error: {balance_error:,.0f} MWh ({balance_pct:.3f}%) - EXCELLENT!")
else:
    print(f"   ⚠️  Balance error: {balance_error:,.0f} MWh ({balance_pct:.2f}%)")

print(f"\n⚡ Chiller Electricity Consumption:")
print(f"   • Total: {chiller_elec.sum():,.0f} MWh_elec")
print(f"   • Average COP: {chiller_heat_removed.sum() / chiller_elec.sum():.2f}")
print(f"   • Expected COP: {CHILLER_COP:.2f}")

if abs(chiller_heat_removed.sum() / chiller_elec.sum() - CHILLER_COP) < 0.1:
    print(f"   ✅ COP matches design value!")
else:
    print(f"   ⚠️  COP deviation detected")

print(f"\n🔥 Dynamic Chiller Load Analysis:")
print(f"   Sample timesteps showing dynamic behavior:")
print(f"   ")
print(f"   {'Timestamp':<20} {'Electro':<10} {'Boosters':<10} {'Chiller':<10} {'Chiller Elec':<12}")
print(f"   {'':20} {'Heat MW':<10} {'Heat MW':<10} {'Heat MW':<10} {'MW':<12}")
print(f"   {'-'*70}")

# Show 10 sample timesteps with varying waste heat utilization
sample_indices = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 8759]
for idx in sample_indices:
    t = n.snapshots[idx]
    e_heat = electro_waste_heat.iloc[idx]
    b_heat = boost_heat_extracted.iloc[idx]
    c_heat = chiller_heat_removed.iloc[idx]
    c_elec = chiller_elec.iloc[idx]
    
    print(f"   {t.strftime('%Y-%m-%d %H:%M'):<20} {e_heat:>8.1f}   {b_heat:>8.1f}   {c_heat:>8.1f}   {c_elec:>10.1f}")

print(f"\n💡 Key Insight:")
print(f"   Chiller Heat = Electrolyser Heat - Booster Consumption")
print(f"   This proves the chiller load varies dynamically with booster utilization!")
print(f"   More boosting → Less chiller thermal → Less electricity consumption")

print(f"\n✅ Verification Summary:")
if balance_pct < 0.1:
    print(f"   ✅ Bus energy balance correct (error < 0.1%)")
else:
    print(f"   ⚠️  Bus energy balance has {balance_pct:.2f}% error")

if boost_heat_extracted.sum() > 0:
    util_fraction = boost_heat_extracted.sum() / electro_waste_heat.sum()
    print(f"   ✅ Boosters consuming {util_fraction*100:.1f}% of waste heat")
    print(f"   ✅ Chiller handling remaining {(1-util_fraction)*100:.1f}%")
    print(f"   ✅ Dynamic load working correctly!")
else:
    print(f"   ⚠️  Boosters not consuming waste heat - check constraints")

print("="*70)



🔍 DYNAMIC CHILLER LOAD VERIFICATION

📊 Annual Energy Balance (electro_60c_bus):
   Heat IN:
      • From electrolyser: 2,507,335 MWh
   
   Heat OUT:
      • To boosters: 142,861 MWh (5.7%)
      • To bypass: 0 MWh (0.0%)
      • To chiller: 1,179,310 MWh (47.0%)
   
   Total OUT: 1,322,171 MWh
   ⚠️  Balance error: 1,185,164 MWh (47.27%)

⚡ Chiller Electricity Consumption:
   • Total: 393,103 MWh_elec
   • Average COP: 3.00
   • Expected COP: 3.00
   ✅ COP matches design value!

🔥 Dynamic Chiller Load Analysis:
   Sample timesteps showing dynamic behavior:
   
   Timestamp            Electro    Boost      Chiller    Chiller Elec
                        Heat MW    Heat MW    Heat MW    MW          
   ----------------------------------------------------------------------
   2027-01-01 00:00        523.4       36.0      240.0         80.0
   2027-02-11 16:00        135.8       44.0       27.6          9.2
   2027-03-25 08:00          0.0        0.0        0.0          0.0
   2027-05-06

In [35]:
# ================================================================
# 4 CHILLER PERFORMANCE
# ================================================================

print("\n" + "="*70)
print("4 CHILLER PERFORMANCE")
print("="*70)

chiller_elec = chiller_thermal / CHILLER_COP
chiller_on_mask = chiller_thermal > 0.1
chiller_hours = chiller_on_mask. sum()

print(f"\n❄️ Operation Statistics:")
print(f"   • Operating hours: {chiller_hours:,} / 8,760 ({chiller_hours/8760*100:.1f}%)")
print(f"   • Total cooling: {chiller_total:,.0f} MWh_thermal")
print(f"   • Total electricity: {chiller_elec. sum():,.0f} MWh_electric")
print(f"   • Actual COP: {chiller_total/chiller_elec.sum():.2f}")

print(f"\n📊 Load Profile (when operating):")
if chiller_hours > 0:
    print(f"   • Average load: {chiller_thermal[chiller_on_mask].mean():.1f} MW_thermal")
    print(f"   • Minimum load: {chiller_thermal[chiller_on_mask]. min():.1f} MW_thermal")
    print(f"   • Maximum load: {chiller_thermal[chiller_on_mask].max():.1f} MW_thermal")
    print(f"   • Capacity: {CHILLER_MW_COOLING:.1f} MW_thermal")
    print(f"   • Peak utilization: {chiller_thermal. max()/CHILLER_MW_COOLING*100:.1f}%")
    print(f"   • Std deviation: {chiller_thermal[chiller_on_mask].std():.1f} MW")

print(f"\n💰 Economics:")
chiller_marginal = n.links. at['chiller', 'marginal_cost']
chiller_opex = chiller_total * chiller_marginal
print(f"   • Marginal cost: {chiller_marginal:.1f} DKK/MWh_thermal")
print(f"   • Total cost: {chiller_opex:,.0f} DKK")
print(f"   • Cost per MWh H2: {chiller_opex/abs(total_h2):.2f} DKK/MWh_H2")

if "electrolyser" in n.links_t.status.columns and "chiller" in n.links_t.status.columns:
    electro_status = n.links_t.status["electrolyser"]
    chiller_status = n.links_t.status["chiller"]
    coupling_errors = ((electro_status > 0.5) & (chiller_status < 0.5)).sum()
    print(f"\n🔗 Coupling Verification:")
    print(f"   • Hours with electrolyser ON, chiller OFF: {coupling_errors}")
    if coupling_errors == 0:
        print(f"   ✅ Perfect coupling verified")
    else:
        print(f"   ⚠️ Coupling constraint violated!")

print("="*70)


4 CHILLER PERFORMANCE

❄️ Operation Statistics:
   • Operating hours: 6,683 / 8,760 (76.3%)
   • Total cooling: 1,179,310 MWh_thermal
   • Total electricity: 393,103 MWh_electric
   • Actual COP: 3.00

📊 Load Profile (when operating):
   • Average load: 176.5 MW_thermal
   • Minimum load: 27.6 MW_thermal
   • Maximum load: 276.0 MW_thermal
   • Capacity: 276.0 MW_thermal
   • Peak utilization: 100.0%
   • Std deviation: 106.1 MW

💰 Economics:
   • Marginal cost: 60.0 DKK/MWh_thermal
   • Total cost: 70,758,615 DKK
   • Cost per MWh H2: 28.22 DKK/MWh_H2

🔗 Coupling Verification:
   • Hours with electrolyser ON, chiller OFF: 0
   ✅ Perfect coupling verified


In [36]:
# ================================================================
# 5 HEAT SUPPLY ANALYSIS
# ================================================================

print("\n" + "="*70)
print("5 HEAT SUPPLY ANALYSIS")
print("="*70)

heat_suppliers = n.links[n.links. bus1 == 'heat_bus']. index
heat_supply_data = []

print(f"\n🔥 Heat Supply Breakdown:")

total_heat_supplied = 0
for supplier in heat_suppliers:
    if supplier in n.links_t. p1.columns:
        heat_output = -n.links_t.p1[supplier]
        total = heat_output.sum()
        total_heat_supplied += total
        
        operating = heat_output > 0.1
        hours = operating.sum()
        
        if total > 100:
            avg = heat_output[operating].mean() if hours > 0 else 0
            max_val = heat_output. max()
            capacity = n.links. at[supplier, 'p_nom'] * n.links.at[supplier, 'efficiency']
            
            heat_supply_data.append({
                'name': supplier,
                'total': total,
                'avg': avg,
                'max': max_val,
                'hours': hours,
                'capacity': capacity
            })

heat_supply_data.sort(key=lambda x: x['total'], reverse=True)

for i, data in enumerate(heat_supply_data, 1):
    pct = (data['total'] / total_heat_supplied * 100) if total_heat_supplied > 0 else 0
    print(f"\n   {i}. {data['name']:30s}")
    print(f"      • Total: {data['total']:>10,.0f} MWh ({pct:>5.1f}%)")
    print(f"      • Average: {data['avg']:>6.1f} MW")
    print(f"      • Maximum: {data['max']:>6.1f} MW")
    print(f"      • Operating hours: {data['hours']:>5,}")
    print(f"      • Capacity: {data['capacity']:>6.1f} MW")

thermal_storage_flow = 0
if 'thermal_storage' in n. stores_t.p. columns:
    thermal_storage_flow = n.stores_t. p['thermal_storage'].sum()

print(f"\n📊 Heat Balance Verification:")
print(f"   • Total supply (links): {total_heat_supplied:,.0f} MWh")
print(f"   • Total demand (load): {total_heat_demand:,.0f} MWh")
print(f"   • Thermal storage net: {thermal_storage_flow:,.0f} MWh")
print(f"   • Balance: {total_heat_supplied - total_heat_demand + thermal_storage_flow:,.0f} MWh")

balance_error_pct = abs(total_heat_supplied - total_heat_demand + thermal_storage_flow) / total_heat_demand * 100
if balance_error_pct < 1:
    print(f"   ✅ Heat balance verified (< 1% error)")
else:
    print(f"   ⚠️ Heat balance error: {balance_error_pct:. 2f}%")

print("="*70)


5 HEAT SUPPLY ANALYSIS

🔥 Heat Supply Breakdown:

   1. chp_heat                      
      • Total:    528,869 MWh ( 42.4%)
      • Average:   63.9 MW
      • Maximum:   65.4 MW
      • Operating hours: 8,280
      • Capacity:   85.4 MW

   2. heat_pump                     
      • Total:    276,521 MWh ( 22.2%)
      • Average:   68.5 MW
      • Maximum:   71.3 MW
      • Operating hours: 4,037
      • Capacity:   71.3 MW

   3. biomass_boiler_boost          
      • Total:    187,689 MWh ( 15.0%)
      • Average:   49.9 MW
      • Maximum:   50.0 MW
      • Operating hours: 3,762
      • Capacity:   50.0 MW

   4. chp_heat_boost                
      • Total:     82,499 MWh (  6.6%)
      • Average:   20.0 MW
      • Maximum:   20.0 MW
      • Operating hours: 4,133
      • Capacity:   20.0 MW

   5. electric_boiler_boost         
      • Total:     69,902 MWh (  5.6%)
      • Average:   19.9 MW
      • Maximum:   20.0 MW
      • Operating hours: 3,508
      • Capacity:   20.0 MW


In [37]:
# ================================================================
# 6 BOOSTER VS DIRECT HEATING COMPARISON
# ================================================================

print("\n" + "="*70)
print("6 BOOSTER VS DIRECT HEATING COMPARISON")
print("="*70)

for component in ['chp_heat', 'electric_boiler', 'gas_boiler', 'biomass_boiler']:
    boost_link = f"{component}_boost"
    
    if component in n.links_t.p1.columns and boost_link in n.links_t.p1.columns:
        
        direct_heat = -n.links_t. p1[component]
        boost_heat = -n.links_t.p1[boost_link]
        
        direct_total = direct_heat.sum()
        boost_total = boost_heat.sum()
        total_from_component = direct_total + boost_total
        
        if total_from_component > 100:
            print(f"\n🔥 {component. upper(). replace('_', ' ')}:")
            print(f"   • Direct heating: {direct_total:,.0f} MWh ({direct_total/total_from_component*100:.1f}%)")
            print(f"   • Boosted heating: {boost_total:,.0f} MWh ({boost_total/total_from_component*100:.1f}%)")
            print(f"   • Total: {total_from_component:,.0f} MWh")
            
            direct_fuel = n.links_t.p0[component]. sum()
            boost_fuel = n.links_t.p0[boost_link].sum()
            
            print(f"\n   ⚡ Fuel/Energy Consumption:")
            print(f"      • Direct: {direct_fuel:,.0f} MWh")
            print(f"      • Boost: {boost_fuel:,.0f} MWh")
            
            equivalent_direct_fuel = boost_total / n.links. at[component, 'efficiency']
            fuel_savings = equivalent_direct_fuel - boost_fuel
            print(f"      • Fuel savings from boosting: {fuel_savings:,.0f} MWh")
            
            marginal_cost = n.links. at[component, 'marginal_cost']
            direct_cost = direct_fuel * marginal_cost
            boost_cost = boost_fuel * marginal_cost
            savings = (equivalent_direct_fuel - boost_fuel) * marginal_cost
            
            print(f"\n   💰 Cost Comparison:")
            print(f"      • Direct cost: {direct_cost:,.0f} DKK")
            print(f"      • Boost cost: {boost_cost:,.0f} DKK")
            print(f"      • Savings from boosting: {savings:,.0f} DKK")
            
            both_running = (direct_heat > 0.1) & (boost_heat > 0.1)
            both_count = both_running.sum()
            
            if both_count > 0:
                print(f"\n   ⚠️ Both running simultaneously: {both_count} hours ({both_count/8760*100:.1f}%)")
            else:
                print(f"\n   ✅ Never both running simultaneously")

print("="*70)


6 BOOSTER VS DIRECT HEATING COMPARISON

🔥 CHP HEAT:
   • Direct heating: 528,869 MWh (86.5%)
   • Boosted heating: 82,499 MWh (13.5%)
   • Total: 611,367 MWh

   ⚡ Fuel/Energy Consumption:
      • Direct: 881,448 MWh
      • Boost: 82,499 MWh
      • Fuel savings from boosting: 54,999 MWh

   💰 Cost Comparison:
      • Direct cost: 175,235,087 DKK
      • Boost cost: 16,401,032 DKK
      • Savings from boosting: 10,934,022 DKK

   ⚠️ Both running simultaneously: 3837 hours (43.8%)

🔥 ELECTRIC BOILER:
   • Direct heating: 43,554 MWh (38.4%)
   • Boosted heating: 69,902 MWh (61.6%)
   • Total: 113,456 MWh

   ⚡ Fuel/Energy Consumption:
      • Direct: 43,994 MWh
      • Boost: 42,365 MWh
      • Fuel savings from boosting: 28,243 MWh

   💰 Cost Comparison:
      • Direct cost: 184,774 DKK
      • Boost cost: 177,933 DKK
      • Savings from boosting: 118,622 DKK

   ⚠️ Both running simultaneously: 653 hours (7.5%)

🔥 GAS BOILER:
   • Direct heating: 0 MWh (0.0%)
   • Boosted heating: 17

In [38]:
# ================================================================
# 7 FUEL CAPACITY COMPLIANCE CHECK
# ================================================================

print("\n" + "="*70)
print("7 FUEL CAPACITY COMPLIANCE CHECK")
print("="*70)

for component in ['chp_heat', 'electric_boiler', 'gas_boiler', 'biomass_boiler']:
    boost_link = f"{component}_boost"
    
    if component not in n.links. index or boost_link not in n.links.index:
        continue
    
    fuel_cap = n.links.at[component, 'p_nom']
    fuel_direct = n.links_t.p0[component]
    fuel_boost = n. links_t.p0[boost_link]
    fuel_total = fuel_direct + fuel_boost
    
    both_on = (fuel_direct > 0.1) & (fuel_boost > 0.1)
    
    if both_on.sum() > 0:
        max_fuel_combined = fuel_total[both_on].max()
        avg_fuel_combined = fuel_total[both_on].mean()
        
        print(f"\n{component.upper().replace('_', ' ')}:")
        print(f"   • Fuel capacity: {fuel_cap:.1f} MW")
        print(f"   • Hours both running: {both_on.sum()} ({both_on.sum()/8760*100:.1f}%)")
        print(f"   • Max combined fuel: {max_fuel_combined:.1f} MW")
        print(f"   • Avg combined fuel: {avg_fuel_combined:.1f} MW")
        
        if max_fuel_combined > fuel_cap * 1.01:
            print(f"   ❌ VIOLATION: Exceeds capacity by {max_fuel_combined - fuel_cap:.1f} MW!")
        else:
            margin = fuel_cap - max_fuel_combined
            print(f"   ✅ Within capacity (margin: {margin:.1f} MW, {margin/fuel_cap*100:.1f}%)")
    else:
        print(f"\n{component.upper().replace('_', ' ')}:")
        print(f"   ✅ Never both running - no capacity check needed")

print("="*70)


7 FUEL CAPACITY COMPLIANCE CHECK

CHP HEAT:
   • Fuel capacity: 142.4 MW
   • Hours both running: 3837 (43.8%)
   • Max combined fuel: 129.1 MW
   • Avg combined fuel: 127.9 MW
   ✅ Within capacity (margin: 13.3 MW, 9.4%)

ELECTRIC BOILER:
   • Fuel capacity: 40.4 MW
   • Hours both running: 653 (7.5%)
   • Max combined fuel: 40.4 MW
   • Avg combined fuel: 37.5 MW
   ✅ Within capacity (margin: -0.0 MW, -0.0%)

GAS BOILER:
   ✅ Never both running - no capacity check needed

BIOMASS BOILER:
   • Fuel capacity: 76.0 MW
   • Hours both running: 519 (5.9%)
   • Max combined fuel: 76.0 MW
   • Avg combined fuel: 71.7 MW
   ✅ Within capacity (margin: -0.0 MW, -0.0%)


In [39]:
# ================================================================
# 8 BOOSTING VS HEAT DEMAND CORRELATION
# ================================================================

print("\n" + "="*70)
print("8 BOOSTING VS HEAT DEMAND CORRELATION")
print("="*70)

import numpy as np

electro_on = n.links_t.p0["electrolyser"] > 1
heat_demand = n.loads_t.p_set["heat_demand"][electro_on]
waste_heat = electro_heat[electro_on]

boost_60c = pd.Series(0, index=heat_demand.index)
for boost in ['chp_heat_boost', 'electric_boiler_boost', 'gas_boiler_boost', 'biomass_boiler_boost']:
    if boost in n.links_t.p2.columns:
        boost_60c += n.links_t. p2[boost][electro_on]. abs()

demand_bins = np.percentile(heat_demand, [0, 25, 50, 75, 100])
demand_labels = ['Q1: Low (0-25%)', 'Q2: Med-Low (25-50%)', 'Q3: Med-High (50-75%)', 'Q4: High (75-100%)']

print(f"\n📊 Boosting vs Heat Demand (When Electrolyser Operating):")

for i, (low, high) in enumerate(zip(demand_bins[:-1], demand_bins[1:])):
    mask = (heat_demand >= low) & (heat_demand < high)
    
    if i == len(demand_bins) - 2:
        mask = (heat_demand >= low) & (heat_demand <= high)
    
    count = mask.sum()
    
    if count > 0:
        avg_demand = heat_demand[mask]. mean()
        avg_waste = waste_heat[mask].mean()
        avg_boost = boost_60c[mask].mean()
        pct_to_boost = (avg_boost / avg_waste * 100) if avg_waste > 0 else 0
        
        print(f"\n   {demand_labels[i]}:")
        print(f"      • Demand range: {low:.0f} - {high:.0f} MW")
        print(f"      • Avg demand: {avg_demand:.1f} MW")
        print(f"      • Avg waste heat: {avg_waste:.1f} MW")
        print(f"      • To boosters: {avg_boost:.1f} MW ({pct_to_boost:.1f}%)")
        print(f"      • To chiller: {avg_waste - avg_boost:.1f} MW ({100-pct_to_boost:.1f}%)")
        print(f"      • Hours: {count:,}")

if len(heat_demand) > 0 and len(boost_60c) > 0:
    correlation = np.corrcoef(heat_demand, boost_60c)[0, 1]
    print(f"\n📈 Statistical Correlation:")
    print(f"   • Correlation coefficient: {correlation:.3f}")
    if correlation > 0.3:
        print(f"   ✅ Positive correlation: Higher demand → More boosting")
    elif correlation < -0.1:
        print(f"   ⚠️ Negative correlation: Higher demand → Less boosting")
    else:
        print(f"   ➖ Weak correlation: Boosting not strongly demand-driven")

print("="*70)


8 BOOSTING VS HEAT DEMAND CORRELATION

📊 Boosting vs Heat Demand (When Electrolyser Operating):

   Q1: Low (0-25%):
      • Demand range: 45 - 68 MW
      • Avg demand: 59.6 MW
      • Avg waste heat: 160.5 MW
      • To boosters: 1.0 MW (0.6%)
      • To chiller: 159.5 MW (99.4%)
      • Hours: 1,671

   Q2: Med-Low (25-50%):
      • Demand range: 68 - 119 MW
      • Avg demand: 88.4 MW
      • Avg waste heat: 190.2 MW
      • To boosters: 13.9 MW (7.3%)
      • To chiller: 176.3 MW (92.7%)
      • Hours: 1,670

   Q3: Med-High (50-75%):
      • Demand range: 119 - 201 MW
      • Avg demand: 165.1 MW
      • Avg waste heat: 222.2 MW
      • To boosters: 31.2 MW (14.0%)
      • To chiller: 191.1 MW (86.0%)
      • Hours: 1,671

   Q4: High (75-100%):
      • Demand range: 201 - 331 MW
      • Avg demand: 240.1 MW
      • Avg waste heat: 218.4 MW
      • To boosters: 39.5 MW (18.1%)
      • To chiller: 179.0 MW (81.9%)
      • Hours: 1,671

📈 Statistical Correlation:
   • Correlation 

In [40]:
# ================================================================
# 9 BOOSTER LINK VERIFICATION
# ================================================================

print("\n" + "="*70)
print("9 BOOSTER LINK VERIFICATION")
print("="*70)

for component in ['chp_heat', 'electric_boiler', 'gas_boiler', 'biomass_boiler']:
    boost_link = f"{component}_boost"
    
    if component not in n.links.index or boost_link not in n.links. index:
        continue
    
    print(f"\n{component.upper().replace('_', ' ')}:")
    
    direct_eff = n.links. at[component, 'efficiency']
    boost_eff = n.links.at[boost_link, 'efficiency']
    boost_eff2 = n.links.at[boost_link, 'efficiency2']
    marginal = n.links.at[component, 'marginal_cost']
    
    print(f"   Link Setup:")
    print(f"      • Direct efficiency: {direct_eff:.3f}")
    print(f"      • Boost efficiency: {boost_eff:.3f} ({'✅ >1' if boost_eff > 1 else '❌ ≤1'})")
    print(f"      • Boost efficiency2: {boost_eff2:.3f}")
    
    direct_hours = (n.links_t.p0[component] > 0.1). sum()
    boost_hours = (n.links_t.p0[boost_link] > 0.1).sum()
    both_hours = ((n.links_t.p0[component] > 0.1) & (n.links_t.p0[boost_link] > 0.1)).sum()
    
    print(f"   Operating Hours:")
    print(f"      • Direct only: {direct_hours - both_hours}")
    print(f"      • Boost only: {boost_hours - both_hours}")
    print(f"      • Both: {both_hours} ({'✅' if both_hours < 100 else '⚠️'})")
    
    if boost_hours > 0:
        boost_on = n.links_t.p0[boost_link] > 0.1
        avg_fuel = n.links_t.p0[boost_link][boost_on].mean()
        avg_heat = (-n.links_t.p1[boost_link][boost_on]).mean()
        cost_per_heat = (avg_fuel * marginal) / avg_heat if avg_heat > 0 else 0
        
        direct_cost_per_heat = marginal / direct_eff
        savings_pct = (1 - cost_per_heat / direct_cost_per_heat) * 100 if direct_cost_per_heat > 0 else 0
        
        print(f"   Economics:")
        print(f"      • Direct cost: {direct_cost_per_heat:.1f} DKK/MWh_heat")
        print(f"      • Boost cost: {cost_per_heat:.1f} DKK/MWh_heat")
        print(f"      • Savings: {savings_pct:.1f}% ({'✅' if savings_pct > 30 else '❌'})")

print("="*70)


9 BOOSTER LINK VERIFICATION

CHP HEAT:
   Link Setup:
      • Direct efficiency: 0.600
      • Boost efficiency: 1.000 (❌ ≤1)
      • Boost efficiency2: -0.400
   Operating Hours:
      • Direct only: 4443
      • Boost only: 296
      • Both: 3837 (⚠️)
   Economics:
      • Direct cost: 331.3 DKK/MWh_heat
      • Boost cost: 198.8 DKK/MWh_heat
      • Savings: 40.0% (✅)

ELECTRIC BOILER:
   Link Setup:
      • Direct efficiency: 0.990
      • Boost efficiency: 1.650 (✅ >1)
      • Boost efficiency2: -0.660
   Operating Hours:
      • Direct only: 1911
      • Boost only: 2855
      • Both: 653 (⚠️)
   Economics:
      • Direct cost: 4.2 DKK/MWh_heat
      • Boost cost: 2.5 DKK/MWh_heat
      • Savings: 40.0% (✅)

GAS BOILER:
   Link Setup:
      • Direct efficiency: 0.800
      • Boost efficiency: 1.333 (✅ >1)
      • Boost efficiency2: -0.533
   Operating Hours:
      • Direct only: 0
      • Boost only: 860
      • Both: 0 (✅)
   Economics:
      • Direct cost: 1390.9 DKK/MWh_heat


In [41]:
# ================================================================
# 10 ECONOMIC SUMMARY
# ================================================================

print("\n" + "="*70)
print("10 ECONOMIC SUMMARY")
print("="*70)

h2_kg = abs(total_h2) / ELECTRO_EFF_H2 * H2_KG_PER_MWH_ELEC
h2_revenue = h2_kg * H2_REVENUE

electro_elec_cost = total_electricity * DSO_TARIFF
electro_opex = h2_kg * ELECTRO_OPEX_VAR_PER_KG
electro_startup_cost = startups * ELECTRO_STARTUP if 'startups' in locals() else 0

chiller_opex_cost = chiller_total * n.links.at['chiller', 'marginal_cost']

total_boost_savings = 0
for component in ['chp_heat', 'electric_boiler', 'gas_boiler', 'biomass_boiler']:
    boost_link = f"{component}_boost"
    if boost_link in n.links_t.p0.columns and component in n.links_t. p1.columns:
        boost_fuel = n.links_t.p0[boost_link]. sum()
        boost_heat = -n.links_t.p1[boost_link].sum()
        
        if boost_heat > 0:
            direct_fuel_equivalent = boost_heat / n.links.at[component, 'efficiency']
            fuel_saved = direct_fuel_equivalent - boost_fuel
            marginal_cost = n.links. at[component, 'marginal_cost']
            savings = fuel_saved * marginal_cost
            total_boost_savings += savings

print(f"\n💰 Electrolyser System Economics (Annual):")
print(f"\n   REVENUE:")
print(f"      • H2 produced: {h2_kg:,.0f} kg")
print(f"      • H2 revenue: {h2_revenue:,.0f} DKK")

print(f"\n   COSTS:")
print(f"      • Electricity (DSO): {electro_elec_cost:,.0f} DKK")
print(f"      • Variable OPEX: {electro_opex:,.0f} DKK")
print(f"      • Startup costs: {electro_startup_cost:,.0f} DKK")
print(f"      • Cooling (chiller OPEX): {chiller_opex_cost:,.0f} DKK")
total_costs = electro_elec_cost + electro_opex + electro_startup_cost + chiller_opex_cost
print(f"      • Total costs: {total_costs:,.0f} DKK")

print(f"\n   SAVINGS FROM WASTE HEAT:")
print(f"      • Boosting savings: {total_boost_savings:,.0f} DKK")
print(f"      • (Avoided direct heating fuel costs)")

net_result = h2_revenue - total_costs

print(f"\n   NET RESULT:")
print(f"      • Gross profit: {net_result:,.0f} DKK")
print(f"      • With boosting benefit: {net_result + total_boost_savings:,.0f} DKK")
print(f"      • Profit margin: {(net_result/h2_revenue)*100:.1f}%")

print(f"\n📊 Key Performance Indicators:")
print(f"      • H2 production cost: {(electro_elec_cost + electro_opex)/h2_kg:.2f} DKK/kg")
print(f"      • Cooling cost per kg H2: {chiller_opex_cost/h2_kg:.2f} DKK/kg")
if total_waste_heat > 0:
    print(f"      • Waste heat value: {total_boost_savings/total_waste_heat:.2f} DKK/MWh_thermal")
print(f"      • System efficiency: {abs(total_h2)/total_electricity*100:.1f}% (LHV)")

print("="*70)


10 ECONOMIC SUMMARY

💰 Electrolyser System Economics (Annual):

   REVENUE:
      • H2 produced: 91,545,984 kg
      • H2 revenue: 3,414,665,197 DKK

   COSTS:
      • Electricity (DSO): 349,656,795 DKK
      • Variable OPEX: 61,335,809 DKK
      • Startup costs: 2,252,920 DKK
      • Cooling (chiller OPEX): 70,758,615 DKK
      • Total costs: 484,004,139 DKK

   SAVINGS FROM WASTE HEAT:
      • Boosting savings: 51,073,084 DKK
      • (Avoided direct heating fuel costs)

   NET RESULT:
      • Gross profit: 2,930,661,058 DKK
      • With boosting benefit: 2,981,734,142 DKK
      • Profit margin: 85.8%

📊 Key Performance Indicators:
      • H2 production cost: 4.49 DKK/kg
      • Cooling cost per kg H2: 0.77 DKK/kg
      • Waste heat value: 38.63 DKK/MWh_thermal
      • System efficiency: 52.3% (LHV)


In [42]:
# ================================================================
# 11 EXECUTIVE SUMMARY DASHBOARD
# ================================================================

print("\n" + "="*70)
print("11 EXECUTIVE SUMMARY DASHBOARD")
print("="*70)

print(f"""
╔══════════════════════════════════════════════════════════════════╗
║                    ELECTROLYSER SYSTEM PERFORMANCE                ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  ⚡ ELECTROLYSER                                                 ║
║     • Operating hours:        {operating_hours:>6,} / 8,760 ({operating_hours/8760*100:>5.1f}%)      ║
║     • H2 produced:            {h2_kg:>12,.0f} kg                     ║
║     • Capacity factor:        {(total_electricity/(ELECTRO_P_NOM*8760))*100:>6.1f}%                         ║
║     • Net profit:             {net_result:>12,.0f} DKK                  ║
║                                                                  ║
║  🌡️ WASTE HEAT UTILIZATION                                      ║
║     • Total waste heat:       {total_waste_heat:>12,.0f} MWh                 ║
║     • To boosting:            {boost_60c_total:>12,.0f} MWh ({boost_60c_total/total_waste_heat*100:>5.1f}%)        ║
║     • To cooling:             {chiller_total:>12,.0f} MWh ({chiller_total/total_waste_heat*100:>5.1f}%)        ║
║     • Boosting savings:       {total_boost_savings:>12,.0f} DKK                  ║
║                                                                  ║
║  ❄️ CHILLER                                                     ║
║     • Operating hours:        {chiller_hours:>6,} / 8,760 ({chiller_hours/8760*100:>5.1f}%)      ║
║     • Avg cooling load:       {chiller_thermal[chiller_on_mask].mean() if chiller_hours > 0 else 0:>6.1f} MW                         ║
║     • Total electricity:      {chiller_elec. sum():>12,.0f} MWh                 ║
║     • Total cost:             {chiller_opex_cost:>12,.0f} DKK                  ║
║                                                                  ║
║  🔥 HEAT SYSTEM                                                  ║
║     • Total demand:           {total_heat_demand:>12,.0f} MWh                 ║
║     • Balance error:          {balance_error_pct:>6.2f}%                         ║
║     • Booster contribution:   {boost_60c_total/total_heat_supplied*100 if total_heat_supplied > 0 else 0:>6.1f}%                         ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
""")

print("="*70)


11 EXECUTIVE SUMMARY DASHBOARD

╔══════════════════════════════════════════════════════════════════╗
║                    ELECTROLYSER SYSTEM PERFORMANCE                ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  ⚡ ELECTROLYSER                                                 ║
║     • Operating hours:         6,683 / 8,760 ( 76.3%)      ║
║     • H2 produced:              91,545,984 kg                     ║
║     • Capacity factor:          54.7%                         ║
║     • Net profit:             2,930,661,058 DKK                  ║
║                                                                  ║
║  🌡️ WASTE HEAT UTILIZATION                                      ║
║     • Total waste heat:          1,322,171 MWh                 ║
║     • To boosting:                 142,861 MWh ( 10.8%)        ║
║     • To cooling:                1,179,310 MWh ( 89.2%)        ║
║     • Boosting

In [ ]:
# ================================================================# 📊 ENERGY BALANCE & SYSTEM OVERVIEW# ================================================================print("="*70)print("📊 ENERGY BALANCE & SYSTEM OVERVIEW")print("="*70)# 1. Waste Heat Bus Energy Balanceprint("\n1️⃣ Waste Heat Bus Energy Balance:")print("-" * 70)electro_waste_heat = n.links_t.p2["electrolyser"].sum()booster_waste_heat = sum([n.links_t.p2[link].sum() for link in                           ["chp_heat_boost", "biomass_boiler_boost",                            "gas_boiler_boost", "electric_boiler_boost"]])chiller_waste_heat = n.links_t.p0["chiller"].sum()bypass_waste_heat = n.links_t.p0["waste_heat_bypass"].sum() if "waste_heat_bypass" in n.links.index else 0print(f"  Waste Heat Produced (Electrolyser):  {electro_waste_heat:>12,.0f} MWh")print(f"  Waste Heat Consumed (Boosters):       {-booster_waste_heat:>12,.0f} MWh ({-booster_waste_heat/electro_waste_heat*100:>5.1f}%)")print(f"  Waste Heat Consumed (Chiller):        {chiller_waste_heat:>12,.0f} MWh ({chiller_waste_heat/electro_waste_heat*100:>5.1f}%)")print(f"  Waste Heat Bypassed:                  {bypass_waste_heat:>12,.0f} MWh ({bypass_waste_heat/electro_waste_heat*100:>5.1f}%)")total_waste_consumed = -booster_waste_heat + chiller_waste_heat + bypass_waste_heatbalance_error = electro_waste_heat - total_waste_consumedbalance_error_pct = abs(balance_error) / electro_waste_heat * 100print(f"\n  Total Consumed:                       {total_waste_consumed:>12,.0f} MWh")print(f"  Balance Error:                        {balance_error:>12,.0f} MWh ({balance_error_pct:.3f}%)")if balance_error_pct < 0.1:    print("  ✅ Energy balance OK (<0.1% error)")else:    print(f"  ⚠️  Energy balance error: {balance_error_pct:.2f}%")# 2. Heat Supply vs Demandprint("\n2️⃣ Heat Supply vs Demand:")print("-" * 70)heat_demand_total = n.loads_t.p.sum().sum()boost_heat = sum([n.links_t.p1[link].sum() for link in                   ["chp_heat_boost", "biomass_boiler_boost",                    "gas_boiler_boost", "electric_boiler_boost"]])direct_heat = sum([n.links_t.p1[link].sum() for link in                    ["chp_heat", "biomass_boiler", "gas_boiler",                     "electric_boiler", "heat_pump"]])storage_heat = n.links_t.p0["storage_discharge"].sum()heat_supply_total = boost_heat + direct_heat + storage_heatprint(f"  Heat Demand:                          {heat_demand_total:>12,.0f} MWh")print(f"  Heat Supply (Boosting):               {boost_heat:>12,.0f} MWh ({boost_heat/heat_supply_total*100:>5.1f}%)")print(f"  Heat Supply (Direct):                 {direct_heat:>12,.0f} MWh ({direct_heat/heat_supply_total*100:>5.1f}%)")print(f"  Heat Supply (Storage):                {storage_heat:>12,.0f} MWh ({storage_heat/heat_supply_total*100:>5.1f}%)")print(f"  Heat Supply (Total):                  {heat_supply_total:>12,.0f} MWh")print(f"  Difference:                           {heat_supply_total - heat_demand_total:>12,.0f} MWh")supply_demand_error = abs(heat_supply_total - heat_demand_total) / heat_demand_total * 100if supply_demand_error < 0.1:    print("  ✅ Supply matches demand (<0.1% error)")else:    print(f"  ⚠️  Supply-demand mismatch: {supply_demand_error:.2f}%")# 3. Electrolyser-Booster-Chiller Couplingprint("\n3️⃣ Electrolyser-Booster-Chiller Coupling:")print("-" * 70)electro_operating_hours = (n.links_t.p0["electrolyser"] > 0).sum()booster_total = sum([n.links_t.p1[link] for link in                      ["chp_heat_boost", "biomass_boiler_boost",                       "gas_boiler_boost", "electric_boiler_boost"]])booster_operating_hours = (booster_total > 0.1).sum()chiller_operating_hours = (n.links_t.p0["chiller"] > 0.1).sum()print(f"  Electrolyser Operating Hours:         {electro_operating_hours:>12,} hrs")print(f"  Booster Operating Hours:              {booster_operating_hours:>12,} hrs")print(f"  Chiller Operating Hours:              {chiller_operating_hours:>12,} hrs")print("\n  ✅ Chiller operates automatically via energy balance:")print("     Electrolyser → Waste Heat → Boosters (priority) → Chiller (residual)")print("\n" + "="*70)